## General information

Just Andrew's Data Munging plus a quick Genetic Programming Model

In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge

In [ ]:
%%time
train = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [ ]:
train_acoustic_data_small = train['acoustic_data'].values[::50]
train_time_to_failure_small = train['time_to_failure'].values[::50]

fig, ax1 = plt.subplots(figsize=(16, 8))
plt.title("Trends of acoustic_data and time_to_failure. 2% of data (sampled)")
plt.plot(train_acoustic_data_small, color='b')
ax1.set_ylabel('acoustic_data', color='b')
plt.legend(['acoustic_data'])
ax2 = ax1.twinx()
plt.plot(train_time_to_failure_small, color='g')
ax2.set_ylabel('time_to_failure', color='g')
plt.legend(['time_to_failure'], loc=(0.875, 0.9))
plt.grid(False)

del train_acoustic_data_small
del train_time_to_failure_small

In [ ]:
# Create a training file with simple derived features
rows = 150_000
segments = int(np.floor(train.shape[0] / rows))

def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

def classic_sta_lta(x, length_sta, length_lta):
    
    sta = np.cumsum(x ** 2)

    # Convert to float
    sta = np.require(sta, dtype=np.float)

    # Copy for LTA
    lta = sta.copy()

    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta

    # Pad zeros
    sta[:length_lta - 1] = 0

    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny

    return sta / lta

X_tr = pd.DataFrame(index=range(segments), dtype=np.float64)

y_tr = pd.DataFrame(index=range(segments), dtype=np.float64, columns=['time_to_failure'])

total_mean = train['acoustic_data'].mean()
total_std = train['acoustic_data'].std()
total_max = train['acoustic_data'].max()
total_min = train['acoustic_data'].min()
total_sum = train['acoustic_data'].sum()
total_abs_sum = np.abs(train['acoustic_data']).sum()

for segment in tqdm_notebook(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = pd.Series(seg['acoustic_data'].values)
    y = seg['time_to_failure'].values[-1]
    
    y_tr.loc[segment, 'time_to_failure'] = y
    X_tr.loc[segment, 'mean'] = x.mean()
    X_tr.loc[segment, 'std'] = x.std()
    X_tr.loc[segment, 'max'] = x.max()
    X_tr.loc[segment, 'min'] = x.min()
    
    
    X_tr.loc[segment, 'mean_change_abs'] = np.mean(np.diff(x))
    X_tr.loc[segment, 'mean_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    X_tr.loc[segment, 'abs_max'] = np.abs(x).max()
    X_tr.loc[segment, 'abs_min'] = np.abs(x).min()
    
    X_tr.loc[segment, 'std_first_50000'] = x[:50000].std()
    X_tr.loc[segment, 'std_last_50000'] = x[-50000:].std()
    X_tr.loc[segment, 'std_first_10000'] = x[:10000].std()
    X_tr.loc[segment, 'std_last_10000'] = x[-10000:].std()
    
    X_tr.loc[segment, 'avg_first_50000'] = x[:50000].mean()
    X_tr.loc[segment, 'avg_last_50000'] = x[-50000:].mean()
    X_tr.loc[segment, 'avg_first_10000'] = x[:10000].mean()
    X_tr.loc[segment, 'avg_last_10000'] = x[-10000:].mean()
    
    X_tr.loc[segment, 'min_first_50000'] = x[:50000].min()
    X_tr.loc[segment, 'min_last_50000'] = x[-50000:].min()
    X_tr.loc[segment, 'min_first_10000'] = x[:10000].min()
    X_tr.loc[segment, 'min_last_10000'] = x[-10000:].min()
    
    X_tr.loc[segment, 'max_first_50000'] = x[:50000].max()
    X_tr.loc[segment, 'max_last_50000'] = x[-50000:].max()
    X_tr.loc[segment, 'max_first_10000'] = x[:10000].max()
    X_tr.loc[segment, 'max_last_10000'] = x[-10000:].max()
    
    X_tr.loc[segment, 'max_to_min'] = x.max() / np.abs(x.min())
    X_tr.loc[segment, 'max_to_min_diff'] = x.max() - np.abs(x.min())
    X_tr.loc[segment, 'count_big'] = len(x[np.abs(x) > 500])
    X_tr.loc[segment, 'sum'] = x.sum()
    
    X_tr.loc[segment, 'mean_change_rate_first_50000'] = np.mean(np.nonzero((np.diff(x[:50000]) / x[:50000][:-1]))[0])
    X_tr.loc[segment, 'mean_change_rate_last_50000'] = np.mean(np.nonzero((np.diff(x[-50000:]) / x[-50000:][:-1]))[0])
    X_tr.loc[segment, 'mean_change_rate_first_10000'] = np.mean(np.nonzero((np.diff(x[:10000]) / x[:10000][:-1]))[0])
    X_tr.loc[segment, 'mean_change_rate_last_10000'] = np.mean(np.nonzero((np.diff(x[-10000:]) / x[-10000:][:-1]))[0])
    
    X_tr.loc[segment, 'q95'] = np.quantile(x, 0.95)
    X_tr.loc[segment, 'q99'] = np.quantile(x, 0.99)
    X_tr.loc[segment, 'q05'] = np.quantile(x, 0.05)
    X_tr.loc[segment, 'q01'] = np.quantile(x, 0.01)
    
    X_tr.loc[segment, 'abs_q95'] = np.quantile(np.abs(x), 0.95)
    X_tr.loc[segment, 'abs_q99'] = np.quantile(np.abs(x), 0.99)
    X_tr.loc[segment, 'abs_q05'] = np.quantile(np.abs(x), 0.05)
    X_tr.loc[segment, 'abs_q01'] = np.quantile(np.abs(x), 0.01)
    
    X_tr.loc[segment, 'trend'] = add_trend_feature(x)
    X_tr.loc[segment, 'abs_trend'] = add_trend_feature(x, abs_values=True)
    X_tr.loc[segment, 'abs_mean'] = np.abs(x).mean()
    X_tr.loc[segment, 'abs_std'] = np.abs(x).std()
    
    X_tr.loc[segment, 'mad'] = x.mad()
    X_tr.loc[segment, 'kurt'] = x.kurtosis()
    X_tr.loc[segment, 'skew'] = x.skew()
    X_tr.loc[segment, 'med'] = x.median()
    
    X_tr.loc[segment, 'Hilbert_mean'] = np.abs(hilbert(x)).mean()
    X_tr.loc[segment, 'Hann_window_mean'] = (convolve(x, hann(150), mode='same') / sum(hann(150))).mean()
    X_tr.loc[segment, 'classic_sta_lta1_mean'] = classic_sta_lta(x, 500, 10000).mean()
    X_tr.loc[segment, 'classic_sta_lta2_mean'] = classic_sta_lta(x, 5000, 100000).mean()
    X_tr.loc[segment, 'classic_sta_lta3_mean'] = classic_sta_lta(x, 3333, 6666).mean()
    X_tr.loc[segment, 'classic_sta_lta4_mean'] = classic_sta_lta(x, 10000, 25000).mean()
    X_tr.loc[segment, 'Moving_average_700_mean'] = x.rolling(window=700).mean().mean(skipna=True)
    X_tr.loc[segment, 'Moving_average_1500_mean'] = x.rolling(window=1500).mean().mean(skipna=True)
    X_tr.loc[segment, 'Moving_average_3000_mean'] = x.rolling(window=3000).mean().mean(skipna=True)
    X_tr.loc[segment, 'Moving_average_6000_mean'] = x.rolling(window=6000).mean().mean(skipna=True)
    ewma = pd.Series.ewm
    X_tr.loc[segment, 'exp_Moving_average_300_mean'] = (ewma(x, span=300).mean()).mean(skipna=True)
    X_tr.loc[segment, 'exp_Moving_average_3000_mean'] = ewma(x, span=3000).mean().mean(skipna=True)
    X_tr.loc[segment, 'exp_Moving_average_30000_mean'] = ewma(x, span=6000).mean().mean(skipna=True)
    no_of_std = 2
    X_tr.loc[segment, 'MA_700MA_std_mean'] = x.rolling(window=700).std().mean()
    X_tr.loc[segment,'MA_700MA_BB_high_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] + no_of_std * X_tr.loc[segment, 'MA_700MA_std_mean']).mean()
    X_tr.loc[segment,'MA_700MA_BB_low_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] - no_of_std * X_tr.loc[segment, 'MA_700MA_std_mean']).mean()
    X_tr.loc[segment, 'MA_400MA_std_mean'] = x.rolling(window=400).std().mean()
    X_tr.loc[segment,'MA_400MA_BB_high_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] + no_of_std * X_tr.loc[segment, 'MA_400MA_std_mean']).mean()
    X_tr.loc[segment,'MA_400MA_BB_low_mean'] = (X_tr.loc[segment, 'Moving_average_700_mean'] - no_of_std * X_tr.loc[segment, 'MA_400MA_std_mean']).mean()
    X_tr.loc[segment, 'MA_1000MA_std_mean'] = x.rolling(window=1000).std().mean()
    
    X_tr.loc[segment, 'iqr'] = np.subtract(*np.percentile(x, [75, 25]))
    X_tr.loc[segment, 'q999'] = np.quantile(x,0.999)
    X_tr.loc[segment, 'q001'] = np.quantile(x,0.001)
    X_tr.loc[segment, 'ave10'] = stats.trim_mean(x, 0.1)
    
    for windows in [10, 100, 1000]:
        x_roll_std = x.rolling(windows).std().dropna().values
        x_roll_mean = x.rolling(windows).mean().dropna().values
        
        X_tr.loc[segment, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X_tr.loc[segment, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X_tr.loc[segment, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X_tr.loc[segment, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X_tr.loc[segment, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X_tr.loc[segment, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X_tr.loc[segment, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X_tr.loc[segment, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X_tr.loc[segment, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X_tr.loc[segment, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X_tr.loc[segment, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        
        X_tr.loc[segment, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X_tr.loc[segment, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X_tr.loc[segment, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X_tr.loc[segment, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X_tr.loc[segment, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X_tr.loc[segment, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X_tr.loc[segment, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X_tr.loc[segment, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X_tr.loc[segment, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
        X_tr.loc[segment, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
        X_tr.loc[segment, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()

In [ ]:
print(f'{X_tr.shape[0]} samples in new train data and {X_tr.shape[1]} columns.')

In [ ]:
np.abs(X_tr.corrwith(y_tr['time_to_failure'])).sort_values(ascending=False).head(12)

In [ ]:
plt.figure(figsize=(44, 24))
cols = list(np.abs(X_tr.corrwith(y_tr['time_to_failure'])).sort_values(ascending=False).head(24).index)
for i, col in enumerate(cols):
    plt.subplot(6, 4, i + 1)
    plt.plot(X_tr[col], color='blue')
    plt.title(col)
    ax1.set_ylabel(col, color='b')

    ax2 = ax1.twinx()
    plt.plot(y_tr, color='g')
    ax2.set_ylabel('time_to_failure', color='g')
    plt.legend([col, 'time_to_failure'], loc=(0.875, 0.9))
    plt.grid(False)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_tr)
X_train_scaled = pd.DataFrame(scaler.transform(X_tr), columns=X_tr.columns)

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
X_test = pd.DataFrame(columns=X_tr.columns, dtype=np.float64, index=submission.index)
plt.figure(figsize=(22, 16))

for i, seg_id in enumerate(tqdm_notebook(X_test.index)):
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    
    x = pd.Series(seg['acoustic_data'].values)
    X_test.loc[seg_id, 'mean'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
        
    X_test.loc[seg_id, 'mean_change_abs'] = np.mean(np.diff(x))
    X_test.loc[seg_id, 'mean_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    X_test.loc[seg_id, 'abs_max'] = np.abs(x).max()
    X_test.loc[seg_id, 'abs_min'] = np.abs(x).min()
    
    X_test.loc[seg_id, 'std_first_50000'] = x[:50000].std()
    X_test.loc[seg_id, 'std_last_50000'] = x[-50000:].std()
    X_test.loc[seg_id, 'std_first_10000'] = x[:10000].std()
    X_test.loc[seg_id, 'std_last_10000'] = x[-10000:].std()
    
    X_test.loc[seg_id, 'avg_first_50000'] = x[:50000].mean()
    X_test.loc[seg_id, 'avg_last_50000'] = x[-50000:].mean()
    X_test.loc[seg_id, 'avg_first_10000'] = x[:10000].mean()
    X_test.loc[seg_id, 'avg_last_10000'] = x[-10000:].mean()
    
    X_test.loc[seg_id, 'min_first_50000'] = x[:50000].min()
    X_test.loc[seg_id, 'min_last_50000'] = x[-50000:].min()
    X_test.loc[seg_id, 'min_first_10000'] = x[:10000].min()
    X_test.loc[seg_id, 'min_last_10000'] = x[-10000:].min()
    
    X_test.loc[seg_id, 'max_first_50000'] = x[:50000].max()
    X_test.loc[seg_id, 'max_last_50000'] = x[-50000:].max()
    X_test.loc[seg_id, 'max_first_10000'] = x[:10000].max()
    X_test.loc[seg_id, 'max_last_10000'] = x[-10000:].max()
    
    X_test.loc[seg_id, 'max_to_min'] = x.max() / np.abs(x.min())
    X_test.loc[seg_id, 'max_to_min_diff'] = x.max() - np.abs(x.min())
    X_test.loc[seg_id, 'count_big'] = len(x[np.abs(x) > 500])
    X_test.loc[seg_id, 'sum'] = x.sum()
    
    X_test.loc[seg_id, 'mean_change_rate_first_50000'] = np.mean(np.nonzero((np.diff(x[:50000]) / x[:50000][:-1]))[0])
    X_test.loc[seg_id, 'mean_change_rate_last_50000'] = np.mean(np.nonzero((np.diff(x[-50000:]) / x[-50000:][:-1]))[0])
    X_test.loc[seg_id, 'mean_change_rate_first_10000'] = np.mean(np.nonzero((np.diff(x[:10000]) / x[:10000][:-1]))[0])
    X_test.loc[seg_id, 'mean_change_rate_last_10000'] = np.mean(np.nonzero((np.diff(x[-10000:]) / x[-10000:][:-1]))[0])
    
    X_test.loc[seg_id, 'q95'] = np.quantile(x,0.95)
    X_test.loc[seg_id, 'q99'] = np.quantile(x,0.99)
    X_test.loc[seg_id, 'q05'] = np.quantile(x,0.05)
    X_test.loc[seg_id, 'q01'] = np.quantile(x,0.01)
    
    X_test.loc[seg_id, 'abs_q95'] = np.quantile(np.abs(x), 0.95)
    X_test.loc[seg_id, 'abs_q99'] = np.quantile(np.abs(x), 0.99)
    X_test.loc[seg_id, 'abs_q05'] = np.quantile(np.abs(x), 0.05)
    X_test.loc[seg_id, 'abs_q01'] = np.quantile(np.abs(x), 0.01)
    
    X_test.loc[seg_id, 'trend'] = add_trend_feature(x)
    X_test.loc[seg_id, 'abs_trend'] = add_trend_feature(x, abs_values=True)
    X_test.loc[seg_id, 'abs_mean'] = np.abs(x).mean()
    X_test.loc[seg_id, 'abs_std'] = np.abs(x).std()
    
    X_test.loc[seg_id, 'mad'] = x.mad()
    X_test.loc[seg_id, 'kurt'] = x.kurtosis()
    X_test.loc[seg_id, 'skew'] = x.skew()
    X_test.loc[seg_id, 'med'] = x.median()
    
    X_test.loc[seg_id, 'Hilbert_mean'] = np.abs(hilbert(x)).mean()
    X_test.loc[seg_id, 'Hann_window_mean'] = (convolve(x, hann(150), mode='same') / sum(hann(150))).mean()
    X_test.loc[seg_id, 'classic_sta_lta1_mean'] = classic_sta_lta(x, 500, 10000).mean()
    X_test.loc[seg_id, 'classic_sta_lta2_mean'] = classic_sta_lta(x, 5000, 100000).mean()
    X_test.loc[seg_id, 'classic_sta_lta3_mean'] = classic_sta_lta(x, 3333, 6666).mean()
    X_test.loc[seg_id, 'classic_sta_lta4_mean'] = classic_sta_lta(x, 10000, 25000).mean()
    X_test.loc[seg_id, 'Moving_average_700_mean'] = x.rolling(window=700).mean().mean(skipna=True)
    X_test.loc[seg_id, 'Moving_average_1500_mean'] = x.rolling(window=1500).mean().mean(skipna=True)
    X_test.loc[seg_id, 'Moving_average_3000_mean'] = x.rolling(window=3000).mean().mean(skipna=True)
    X_test.loc[seg_id, 'Moving_average_6000_mean'] = x.rolling(window=6000).mean().mean(skipna=True)
    ewma = pd.Series.ewm
    X_test.loc[seg_id, 'exp_Moving_average_300_mean'] = (ewma(x, span=300).mean()).mean(skipna=True)
    X_test.loc[seg_id, 'exp_Moving_average_3000_mean'] = ewma(x, span=3000).mean().mean(skipna=True)
    X_test.loc[seg_id, 'exp_Moving_average_30000_mean'] = ewma(x, span=6000).mean().mean(skipna=True)
    no_of_std = 2
    X_test.loc[seg_id, 'MA_700MA_std_mean'] = x.rolling(window=700).std().mean()
    X_test.loc[seg_id,'MA_700MA_BB_high_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X_test.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X_test.loc[seg_id,'MA_700MA_BB_low_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X_test.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X_test.loc[seg_id, 'MA_400MA_std_mean'] = x.rolling(window=400).std().mean()
    X_test.loc[seg_id,'MA_400MA_BB_high_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X_test.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X_test.loc[seg_id,'MA_400MA_BB_low_mean'] = (X_test.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X_test.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X_test.loc[seg_id, 'MA_1000MA_std_mean'] = x.rolling(window=1000).std().mean()
    
    X_test.loc[seg_id, 'iqr'] = np.subtract(*np.percentile(x, [75, 25]))
    X_test.loc[seg_id, 'q999'] = np.quantile(x,0.999)
    X_test.loc[seg_id, 'q001'] = np.quantile(x,0.001)
    X_test.loc[seg_id, 'ave10'] = stats.trim_mean(x, 0.1)
    
    for windows in [10, 100, 1000]:
        x_roll_std = x.rolling(windows).std().dropna().values
        x_roll_mean = x.rolling(windows).mean().dropna().values
        
        X_test.loc[seg_id, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X_test.loc[seg_id, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X_test.loc[seg_id, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X_test.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X_test.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X_test.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X_test.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X_test.loc[seg_id, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X_test.loc[seg_id, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X_test.loc[seg_id, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X_test.loc[seg_id, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        
        X_test.loc[seg_id, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X_test.loc[seg_id, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X_test.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X_test.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X_test.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X_test.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X_test.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X_test.loc[seg_id, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X_test.loc[seg_id, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
        X_test.loc[seg_id, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
        X_test.loc[seg_id, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()
    
    if i < 12:
        plt.subplot(6, 4, i + 1)
        plt.plot(seg['acoustic_data'])
        plt.title(seg_id)
    
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [ ]:
X_train_scaled.head()

In [ ]:
def GPI(data):
    return (5.612045 +
            0.0399999991*np.tanh(((((((((((((data["q05_roll_mean_10"]) - (((data["q95"]) + ((((data["iqr"]) + (((data["q01_roll_std_100"]) + (((((data["q95_roll_std_10"]) + (data["q05_roll_std_1000"]))) + (((0.3183098733) * 2.0)))))))/2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((10.64747142791748047)) * ((((10.64747524261474609)) * (((data["q05"]) - ((((data["q95"]) + (((data["q05_roll_std_100"]) + (((((((((data["iqr"]) + (((data["q05_roll_std_100"]) * 2.0)))/2.0)) + ((1.0)))) + (data["q95_roll_mean_10"]))/2.0)))))/2.0)))))))) +
            0.0399999991*np.tanh((((11.80052089691162109)) * ((((11.80052089691162109)) * (((((data["q99"]) + (data["q05"]))) - (((data["q05_roll_std_1000"]) + (((((data["iqr"]) + (((((((data["ave_roll_std_100"]) + (((data["q05_roll_std_10"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)))))))))) +
            0.0399999991*np.tanh(((((((((((data["q05"]) - (((1.0) + (((data["iqr"]) - (((((((((((data["q05"]) - (data["q05_roll_std_100"]))) - (data["q95"]))) * 2.0)) * 2.0)) + (data["q01_roll_mean_100"]))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((10.65805721282958984)) * ((((((((9.97931766510009766)) * (((((((data["q05_roll_mean_10"]) - (((((data["q95"]) + (((data["q05_roll_std_10"]) * 2.0)))) * 2.0)))) - (data["iqr"]))) - (data["q01_roll_std_100"]))))) - (data["iqr"]))) - (data["q01_roll_std_100"]))))) +
            0.0399999991*np.tanh((((11.62679195404052734)) * (((data["q05_roll_mean_10"]) + ((((11.62679195404052734)) * (((data["iqr"]) + (((((((((data["q05"]) - (((data["q05_roll_std_10"]) * 2.0)))) * 2.0)) - (((data["iqr"]) + (data["abs_std"]))))) * 2.0)))))))))) +
            0.0399999991*np.tanh(((((data["av_change_abs_roll_std_1000"]) * 2.0)) - ((((11.25893974304199219)) * ((((11.25893974304199219)) * ((((11.25893974304199219)) * (((data["q05_roll_std_100"]) + (((data["q05_roll_std_1000"]) + (((((data["MA_700MA_BB_high_mean"]) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0)))) * 2.0)))))))))))))) +
            0.0399999991*np.tanh(((((((((-1.0) - (((((((data["iqr"]) + (((((data["q05_roll_std_10"]) + (((data["q05_roll_std_1000"]) + (data["std_roll_std_100"]))))) * 2.0)))) + (data["q05_roll_std_10"]))) * (((data["av_change_abs_roll_mean_10"]) + ((14.63721561431884766)))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((((((((((((((data["q05"]) - (((data["q05_roll_std_10"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) - (((data["iqr"]) + (((data["q05_roll_std_100"]) + (data["min_roll_std_100"]))))))) * 2.0)) * 2.0)) * 2.0)) + (data["iqr"]))) * 2.0)) +
            0.0399999991*np.tanh((((-1.0*((((((((((((((((data["iqr"]) + (((((((((data["q05_roll_std_10"]) * 2.0)) + (((data["q01_roll_std_10"]) * 2.0)))) * 2.0)) + (data["q05_roll_std_10"]))))) * 2.0)) * 2.0)) * 2.0)) + (data["std_first_10000"]))) * 2.0)) * 2.0))))) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["min_last_50000"]) - (((((((((((((((((data["q95"]) + (((data["q05_roll_std_10"]) * ((7.0)))))) * 2.0)) - (data["min_last_50000"]))) * 2.0)) - (data["min_first_10000"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((data["min_roll_std_100"]) - ((((13.68760871887207031)) * (((((((data["q05_roll_std_1000"]) + ((((((data["min_roll_std_100"]) + ((((data["q95_roll_mean_10"]) + (data["av_change_rate_roll_mean_1000"]))/2.0)))/2.0)) + (((((((data["q95_roll_mean_10"]) + (0.3183098733))) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)))))) +
            0.0399999991*np.tanh(((((((((((((((((data["q01_roll_std_100"]) - (((((((((data["q05_roll_std_1000"]) + (data["ave_roll_std_10"]))) + (((data["iqr"]) - (data["q05_roll_mean_10"]))))) * 2.0)) * ((7.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((9.0)) * ((((9.17904281616210938)) * ((((9.17904281616210938)) * ((((9.0)) * (((data["MA_700MA_BB_low_mean"]) - (((data["q05_roll_std_10"]) + (np.tanh((((((data["q95"]) * 2.0)) + (np.tanh(((9.17903995513916016)))))))))))))))))))) +
            0.0399999991*np.tanh((((8.0)) * ((((((((((-1.0*((((data["min_roll_std_100"]) + (((data["av_change_abs_roll_std_10"]) + (((data["q05_roll_std_1000"]) + ((((((6.26586675643920898)) * (data["q05_roll_std_100"]))) * 2.0))))))))))) * 2.0)) - (data["max_roll_std_100"]))) * 2.0)) - (data["av_change_abs_roll_std_10"]))))) +
            0.0399999991*np.tanh(((((((data["av_change_rate_roll_mean_10"]) + ((9.27850341796875000)))) - ((((9.27850341796875000)) * (((((((((((((((data["q05_roll_std_100"]) - (((data["q05_roll_mean_10"]) - (data["iqr"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (data["q05_roll_std_100"]))))))) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["mean_change_rate_last_50000"]) + ((((((((((-1.0*((1.0)))) - ((((((((((((data["q95_roll_mean_10"]) + (data["avg_first_50000"]))/2.0)) / 2.0)) + (data["q05_roll_std_100"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((11.44563198089599609)) * ((((11.44563579559326172)) * (((data["min_last_50000"]) - (((((((((((data["q05_roll_std_10"]) * 2.0)) - (data["q05_roll_mean_10"]))) * 2.0)) * 2.0)) + (((((((data["q05_roll_std_10"]) + (data["q05_roll_mean_10"]))/2.0)) + (data["q05_roll_mean_10"]))/2.0)))))))))) +
            0.0399999991*np.tanh(((((((((((((data["q05_roll_std_100"]) + (data["q05"]))) - (((((data["q95"]) + (((data["Moving_average_6000_mean"]) + (((((((data["q05_roll_std_100"]) + (data["iqr"]))) * 2.0)) - (data["q05"]))))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((9.0)) * ((((9.0)) * ((((9.0)) * ((((((8.0)) * (((data["q05_roll_mean_10"]) + (((data["exp_Moving_average_300_mean"]) - ((((4.99870872497558594)) * (((data["q05_roll_std_10"]) * 2.0)))))))))) - (((data["q05_roll_mean_10"]) * 2.0)))))))))) +
            0.0399999991*np.tanh((((8.78560256958007812)) * ((((8.78560256958007812)) * (((np.tanh((np.tanh(((((((((8.78560256958007812)) * (((((-2.0) * (data["q95_roll_mean_10"]))) - ((((data["q05_roll_std_1000"]) + (data["iqr"]))/2.0)))))) * 2.0)) * 2.0)))))) - (data["q05_roll_std_1000"]))))))) +
            0.0399999991*np.tanh(((((((((((((((data["std_roll_mean_1000"]) - (data["std_last_10000"]))) - ((((14.58706188201904297)) * ((((data["q05_roll_std_100"]) + (((data["q01_roll_mean_100"]) * ((((data["Moving_average_6000_mean"]) + (data["q05_roll_mean_10"]))/2.0)))))/2.0)))))) - (data["abs_max_roll_mean_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["q95_roll_mean_10"]) - (((((((((data["iqr"]) + (((data["q01_roll_std_100"]) - (np.tanh(((((-1.0*((data["iqr"])))) - (data["Moving_average_6000_mean"]))))))))) + (((data["q95"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((data["max_to_min"]) - (((data["avg_first_50000"]) + ((((9.91363239288330078)) * ((((((9.91363239288330078)) * (((((data["q05_roll_std_10"]) + (np.tanh((np.tanh(((((0.44413101673126221)) + ((((data["q95"]) + (data["avg_first_50000"]))/2.0)))))))))) * 2.0)))) * 2.0)))))))) +
            0.0399999991*np.tanh(((((((((((((((((((data["Moving_average_1500_mean"]) - (((data["max_roll_mean_100"]) + (((data["q05_roll_std_10"]) * (((((10.65131282806396484)) + ((7.0)))/2.0)))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["q05_roll_std_10"]) * 2.0)) * 2.0)) * (data["q05_roll_mean_100"]))) - (((((((((((((((data["std_roll_std_10"]) + (data["q05_roll_mean_100"]))) + (((((data["q05_roll_std_10"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) + (data["abs_q05"]))) * 2.0)))) +
            0.0399999991*np.tanh(((((((((data["q05_roll_std_100"]) - (((((np.tanh((data["q95_roll_mean_1000"]))) + (((data["q01_roll_std_1000"]) + (((((((data["q05_roll_std_10"]) + (((((data["q05_roll_std_100"]) * 2.0)) * 2.0)))) * 2.0)) + ((4.0)))))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((-3.0) * (((data["av_change_rate_roll_std_100"]) + (((((((((data["sum"]) + (((((data["q05_roll_mean_100"]) + (((((((data["q95"]) + (data["q05_roll_std_100"]))) + (data["q05_roll_std_100"]))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))))) * 2.0)) +
            0.0399999991*np.tanh((((((((((((((((-1.0*((((((((data["q05_roll_std_1000"]) - ((((np.tanh((data["q01_roll_mean_100"]))) + ((-1.0*((((data["q95"]) * 2.0))))))/2.0)))) * 2.0)) * 2.0))))) - (3.0))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((-1.0*((((((((((((((((((data["q05_roll_std_100"]) * 2.0)) + ((((data["Moving_average_3000_mean"]) + ((((data["abs_max_roll_std_1000"]) + ((((data["q01_roll_mean_1000"]) + (data["abs_q05"]))/2.0)))/2.0)))/2.0)))) + (data["q95"]))) + (data["std_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0))))) +
            0.0399999991*np.tanh(((((((((((-2.0) - (data["q05_roll_std_100"]))) + (((((-2.0) * (((((data["q05_roll_std_100"]) + (((data["q05_roll_std_10"]) - (((data["q95_roll_mean_1000"]) - (((data["q05_roll_std_10"]) * 2.0)))))))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((((((((-1.0*((((data["q05_roll_std_100"]) + (((((data["q95_roll_mean_1000"]) * (np.tanh((np.tanh((((((np.tanh((data["q01_roll_std_10"]))) * 2.0)) + (data["q05_roll_std_100"]))))))))) + (((data["q01_roll_std_10"]) * 2.0))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((-1.0*((((((((((((np.tanh((((((((data["iqr"]) + ((((data["q05_roll_mean_1000"]) + (data["q95_roll_mean_10"]))/2.0)))) * 2.0)) * 2.0)))) + (((data["q05_roll_std_1000"]) + (data["q95"]))))) * 2.0)) * 2.0)) + (data["q95"]))) * 2.0))))) * 2.0)) +
            0.0399999991*np.tanh((((((-1.0*(((((((4.0)) - (((((data["q05"]) - (((((data["q05_roll_std_100"]) * 2.0)) * 2.0)))) * 2.0)))) * 2.0))))) - (((((data["avg_last_10000"]) * 2.0)) + (((((data["avg_last_10000"]) * (data["q05_roll_mean_100"]))) * 2.0)))))) * 2.0)) +
            0.0399999991*np.tanh((((((((((((((-1.0*((((data["q05_roll_std_1000"]) + ((((((((data["Moving_average_6000_mean"]) + (data["q01_roll_std_10"]))/2.0)) + (data["q95_roll_mean_10"]))) + (data["q95_roll_mean_10"])))))))) * 2.0)) * 2.0)) - ((((data["abs_q05"]) + (data["q05_roll_std_1000"]))/2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((((-1.0*((((((((((((((data["q01_roll_std_100"]) + (((data["MA_1000MA_std_mean"]) + (((((data["q05_roll_std_1000"]) + (0.3183098733))) * 2.0)))))) - (data["Moving_average_3000_mean"]))) * 2.0)) + (data["iqr"]))) * 2.0)) + (data["MA_1000MA_std_mean"])))))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((((((data["abs_q95"]) - (((data["abs_max_roll_std_10"]) - (((((data["q05_roll_std_100"]) + (((data["q95"]) + (((data["ave10"]) / 2.0)))))) * (((data["q05_roll_std_100"]) - ((10.0)))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((((12.41961765289306641)) * (data["classic_sta_lta1_mean"]))) - ((((12.41961383819580078)) * ((((12.41961765289306641)) * ((((12.41961765289306641)) * ((((((np.tanh((data["q95_roll_std_10"]))) + (data["q01_roll_std_100"]))/2.0)) + (((data["q05_roll_std_1000"]) - (np.tanh((-2.0))))))))))))))) +
            0.0399999991*np.tanh((((-1.0*((((((((data["std_roll_std_1000"]) + (((((3.0) + (data["Moving_average_700_mean"]))) * (((np.tanh((((data["q05_roll_std_100"]) + (((data["Moving_average_6000_mean"]) + (np.tanh((((data["q05_roll_std_1000"]) * 2.0)))))))))) * 2.0)))))) * 2.0)) * 2.0))))) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["iqr"]) - (((((((np.tanh((((((data["std"]) * 2.0)) + (((data["q05_roll_std_100"]) * 2.0)))))) * (data["sum"]))) + (((((data["MA_700MA_BB_high_mean"]) + (data["q05_roll_std_100"]))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["min_roll_std_100"]) + (((((((((data["med"]) - (data["min_roll_std_100"]))) - (((1.0) - (((((data["exp_Moving_average_30000_mean"]) - (((((data["q05_roll_std_10"]) * 2.0)) * 2.0)))) * 2.0)))))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["q05_roll_std_100"]) * 2.0)) + (((data["q05_roll_std_100"]) + ((((-1.0*((((data["q05_roll_std_100"]) * (data["q05_roll_std_100"])))))) * ((((13.66018199920654297)) * (((((data["q05_roll_std_100"]) + (np.tanh((data["abs_q05"]))))) * 2.0)))))))))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((data["q05_roll_std_100"]) - (((((((data["q05_roll_std_100"]) + (data["q05_roll_std_10"]))) + (data["Moving_average_3000_mean"]))) * (((((((((((3.0) + (data["q05_roll_std_10"]))) * (((data["q05_roll_std_100"]) + (data["Moving_average_1500_mean"]))))) + ((8.0)))) * 2.0)) * 2.0)))))) +
            0.0399999991*np.tanh(((((((((((((-3.0) - (np.tanh((data["q05_roll_std_1000"]))))) + (((((data["Moving_average_6000_mean"]) - (((((data["q05_roll_std_1000"]) + (np.tanh((((data["q05_roll_std_1000"]) + (data["iqr"]))))))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((data["iqr"]) - (((((((((((data["q95"]) * 2.0)) + (np.tanh((((((data["q05_roll_std_10"]) + (data["q05_roll_std_1000"]))) + (((data["q05_roll_std_10"]) - (((data["exp_Moving_average_3000_mean"]) * (data["exp_Moving_average_300_mean"]))))))))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) +
            0.0399999991*np.tanh(((((((((((((data["abs_q05"]) - (((((data["q05_roll_std_100"]) * 2.0)) + (data["mean_change_rate_last_50000"]))))) + (((((((-2.0) - (((((data["q05_roll_std_100"]) * 2.0)) + (data["q05_roll_std_100"]))))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((data["Moving_average_3000_mean"]) - (((((3.1415927410) + (data["ave_roll_mean_10"]))) * ((((((14.16982841491699219)) * (((((((data["ave_roll_mean_10"]) + (((data["q95"]) * 2.0)))) - (np.tanh((((data["Moving_average_3000_mean"]) - (data["q05_roll_std_100"]))))))) * 2.0)))) * 2.0)))))) +
            0.0399999991*np.tanh(((((((-3.0) + (((((-3.0) + (((((((((data["exp_Moving_average_30000_mean"]) - (((((data["q05_roll_std_100"]) * 2.0)) + (data["iqr"]))))) * 2.0)) - (data["q01_roll_std_1000"]))) * 2.0)))) * 2.0)))) - (data["exp_Moving_average_30000_mean"]))) * 2.0)) +
            0.0399999991*np.tanh((((((((10.0)) * (((data["MA_700MA_BB_low_mean"]) - (((((((data["q05_roll_std_10"]) + (data["abs_q05"]))) + (((((((data["q05_roll_std_10"]) + (((data["q05_roll_std_10"]) + (data["Moving_average_1500_mean"]))))) * 2.0)) * 2.0)))) * 2.0)))))) + (data["abs_q05"]))) * 2.0)) +
            0.0399999991*np.tanh((((13.64070606231689453)) * (((data["q05"]) + ((((((((((((((13.64070606231689453)) * (((np.tanh((data["q99_roll_mean_1000"]))) - (data["q05_roll_std_100"]))))) + (data["q95_roll_mean_100"]))/2.0)) - (data["iqr"]))) + (data["q95_roll_mean_100"]))/2.0)) - (data["iqr"]))))))) +
            0.0399999991*np.tanh(((((((((data["classic_sta_lta1_mean"]) + (((((((((((data["ave_roll_mean_100"]) * (data["exp_Moving_average_3000_mean"]))) + (((data["q95_roll_std_100"]) - (((((data["q05_roll_std_10"]) + (((data["q95"]) * 2.0)))) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((11.20326423645019531)) * ((((((11.20326423645019531)) * ((((((11.20326423645019531)) * (((((((data["q05"]) - (1.0))) - (np.tanh((data["q05_roll_std_100"]))))) * 2.0)))) - (data["av_change_abs_roll_std_100"]))))) - (((1.0) - (data["av_change_abs_roll_std_100"]))))))) +
            0.0399999991*np.tanh(((((((((data["q05_roll_mean_1000"]) + (data["q05_roll_mean_1000"]))) * (data["exp_Moving_average_30000_mean"]))) - ((((((((((-1.0) + (((((data["q05_roll_mean_1000"]) + (data["q05_roll_std_10"]))) + (((data["q05_roll_std_10"]) * 2.0)))))/2.0)) + (data["q05_roll_std_100"]))) * 2.0)) * 2.0)))) * 2.0)) +
            0.0399999991*np.tanh(((((((((((((data["sum"]) - (((((((((data["q01_roll_std_1000"]) * 2.0)) + (((((data["q05_roll_std_10"]) + (((data["q01_roll_std_100"]) - (-2.0))))) * 2.0)))) * 2.0)) + (data["q01_roll_std_100"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((((((((data["min_last_50000"]) - (((((data["q95"]) - (((((((data["Moving_average_3000_mean"]) * (data["Moving_average_3000_mean"]))) - (((data["q05_roll_std_10"]) * 2.0)))) * 2.0)))) + (data["q95_roll_mean_10"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((data["q01_roll_std_1000"]) + (((((((data["abs_q05"]) - (((((((np.tanh((((((3.0) + (((((data["q05_roll_std_1000"]) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)) - (data["q05"]))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((((((((-3.0) * (((data["q05_roll_std_100"]) + (((data["q05_roll_mean_100"]) + (data["q05_roll_std_100"]))))))) - (((data["abs_max"]) * 2.0)))) - (((data["abs_q05"]) * (data["q95_roll_std_1000"]))))) * 2.0)) - (-3.0))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((((np.tanh((((((data["ave_roll_mean_1000"]) - (((((data["q01_roll_std_10"]) - (((((data["abs_q05"]) * (((data["q95_roll_mean_10"]) - (data["iqr"]))))) * 2.0)))) * 2.0)))) * 2.0)))) * 2.0)) - (data["q01_roll_std_10"]))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((((6.0)) * (((((((np.tanh((((((data["max_to_min"]) + (data["mean_change_rate_first_10000"]))) + (data["q01_roll_std_1000"]))))) + ((-1.0*((((((((data["q05_roll_std_100"]) - (np.tanh((data["iqr"]))))) * 2.0)) * 2.0))))))) * 2.0)) * 2.0)))) * 2.0)) +
            0.0399999991*np.tanh(((((((((-3.0) * (((data["abs_q05"]) + (((((data["q05_roll_std_10"]) * 2.0)) - (np.tanh((((((((data["q05_roll_std_100"]) + (((data["abs_q05"]) + (((data["q05_roll_std_10"]) * 2.0)))))) * 2.0)) * 2.0)))))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((((((data["ave_roll_mean_100"]) - (((data["q05_roll_std_10"]) + (((((data["q01_roll_std_10"]) + (((((((data["q05_roll_std_1000"]) - (np.tanh((((data["q05_roll_std_1000"]) * 2.0)))))) * 2.0)) * 2.0)))) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((data["exp_Moving_average_30000_mean"]) - (((((np.tanh((((np.tanh((((data["max_roll_std_1000"]) + (np.tanh((((((((((data["q95_roll_mean_100"]) + (data["q05_roll_std_100"]))) * 2.0)) * 2.0)) - (data["classic_sta_lta3_mean"]))))))))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((((8.0)) * ((((((((7.0)) * ((((((8.0)) * (((data["q05"]) - (((((3.0) / 2.0)) + (data["q05_roll_std_100"]))))))) + (data["max_roll_std_10"]))))) + (data["q05"]))) + (data["mean_change_abs"]))))) * 2.0)) +
            0.0399999991*np.tanh(((((((((((data["min_last_50000"]) - (data["q05_roll_std_1000"]))) - ((((((((((data["q05_roll_std_1000"]) + (data["abs_q05"]))/2.0)) - (((((data["exp_Moving_average_3000_mean"]) * (data["exp_Moving_average_3000_mean"]))) - (((data["iqr"]) * 2.0)))))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((11.31315994262695312)) * (((((((np.tanh((((np.tanh(((((11.31315994262695312)) * (((data["q95_roll_mean_1000"]) + ((((11.31315994262695312)) * (data["q05_roll_std_10"]))))))))) - (data["q05_roll_std_100"]))))) * 2.0)) * 2.0)) - (((data["q05_roll_std_10"]) + ((0.67717570066452026)))))))) +
            0.0399999991*np.tanh(((((((((((((((((data["q05"]) + (((data["MA_400MA_std_mean"]) * (((((data["exp_Moving_average_3000_mean"]) - (((data["q01_roll_std_10"]) * 2.0)))) * (((data["q01_roll_std_10"]) + (data["exp_Moving_average_3000_mean"]))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((-1.0*((((((((((((((data["q05_roll_std_1000"]) + (np.tanh((((data["Moving_average_700_mean"]) - (((data["q01_roll_std_10"]) * ((-1.0*(((((4.22244548797607422)) * (((data["ave10"]) + (data["q05_roll_std_10"])))))))))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0))))) +
            0.0399999991*np.tanh(((data["min_roll_std_10"]) - (((((((((((((((((data["q01_roll_std_10"]) + (((data["iqr"]) - (data["q95_roll_mean_1000"]))))) * 2.0)) + (data["iqr"]))) * 2.0)) * 2.0)) + (data["q01_roll_std_10"]))) + (np.tanh((data["avg_last_50000"]))))) * 2.0)))) +
            0.0399999991*np.tanh(((data["min_roll_mean_1000"]) - ((((12.11624908447265625)) * (((((((data["q05_roll_std_100"]) * (data["abs_q05"]))) + ((((12.11624908447265625)) * (((data["ave_roll_mean_100"]) + (((((data["q05_roll_std_100"]) - (data["min_roll_mean_1000"]))) - (data["min_roll_mean_1000"]))))))))) * 2.0)))))) +
            0.0399999991*np.tanh(((((((data["skew"]) - (((((((((data["q01_roll_std_100"]) - (-1.0))) * 2.0)) * 2.0)) + (((((((((((((data["q05_roll_std_1000"]) - (-1.0))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["abs_q05"]))))))) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((((data["min"]) - (np.tanh((np.tanh((((((((((data["q05_roll_std_100"]) - (0.3183098733))) * 2.0)) - (((data["Moving_average_6000_mean"]) * (((data["ave_roll_mean_1000"]) + (((data["av_change_abs_roll_mean_10"]) / 2.0)))))))) * 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh(((((((data["q01_roll_std_10"]) * (data["q05_roll_std_100"]))) * (((((data["q95_roll_std_100"]) + ((((6.67154550552368164)) * (((((-1.0) - (data["q05_roll_std_100"]))) * 2.0)))))) * 2.0)))) - (((((data["av_change_abs_roll_mean_10"]) - (data["q95_roll_std_100"]))) - (data["avg_last_10000"]))))) +
            0.0399999991*np.tanh((-1.0*((((((((data["q01_roll_std_10"]) - (data["mean_change_rate_first_10000"]))) + (((data["av_change_abs_roll_mean_10"]) + (((data["q95_roll_mean_10"]) - (((data["abs_q05"]) * ((-1.0*((((data["q01_roll_std_1000"]) + (data["q95_roll_mean_10"])))))))))))))) * (((data["q01_roll_std_100"]) * (data["q01_roll_std_100"])))))))) +
            0.0399999991*np.tanh(((((((((((((data["mean"]) + (data["q05_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (((data["q01_roll_std_10"]) * ((((-1.0*((data["q95_roll_mean_100"])))) - ((((data["q01_roll_std_10"]) + (((((data["q01_roll_std_10"]) * 2.0)) + (3.0))))/2.0)))))))) +
            0.0399999991*np.tanh(((((((((data["avg_last_10000"]) - (((data["q05_roll_std_10"]) - (((((data["abs_q05"]) - (((((((((((data["q05_roll_std_10"]) - (np.tanh((((data["q05_roll_std_10"]) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((((((((-1.0*((((((((((data["abs_q05"]) + (data["q95"]))) - (np.tanh((np.tanh((data["skew"]))))))) * 2.0)) * 2.0))))) - (((data["av_change_abs_roll_mean_10"]) + (((data["abs_q05"]) + (data["abs_q05"]))))))) * 2.0)) * 2.0)) * 2.0)) +
            0.0399999991*np.tanh((((14.75798225402832031)) * (((((data["q05_roll_mean_10"]) - ((2.0)))) - ((((14.75798606872558594)) * (((data["q05_roll_std_10"]) - (np.tanh((((((((data["q05_roll_std_10"]) * 2.0)) * 2.0)) - (((((data["q05_roll_std_10"]) * 2.0)) - (data["abs_q05"]))))))))))))))) +
            0.0399999991*np.tanh((((-1.0*((((data["mean_change_rate"]) + (((data["q05_roll_std_1000"]) + (((data["q05_roll_std_100"]) * ((((((data["q05_roll_std_100"]) * (((data["exp_Moving_average_30000_mean"]) + (data["q05_roll_std_10"]))))) + (((data["q05_roll_std_1000"]) - (((data["exp_Moving_average_300_mean"]) * (data["exp_Moving_average_300_mean"]))))))/2.0))))))))))) * 2.0)) +
            0.0399999991*np.tanh((((((((data["mean_change_rate_first_10000"]) + (data["q01_roll_std_10"]))/2.0)) - (data["av_change_abs_roll_mean_10"]))) - (((data["av_change_rate_roll_mean_1000"]) + (((((((((data["q01_roll_std_10"]) - (np.tanh((((data["q99_roll_mean_10"]) * ((((9.77703285217285156)) + (data["q01_roll_std_10"]))))))))) * 2.0)) * 2.0)) * 2.0)))))) +
            0.0399999991*np.tanh(((((((data["q05_roll_mean_1000"]) * ((((data["exp_Moving_average_30000_mean"]) + (((data["Moving_average_1500_mean"]) * (data["Moving_average_3000_mean"]))))/2.0)))) - (((((np.tanh((((((((data["q95"]) * 2.0)) - (((data["exp_Moving_average_30000_mean"]) * (data["Moving_average_3000_mean"]))))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) +
            0.0399999991*np.tanh(((data["skew"]) - (((data["q95"]) + (((((((((data["q95"]) + (data["av_change_abs_roll_mean_10"]))) + ((((((data["iqr"]) + (data["av_change_abs_roll_mean_100"]))) + (data["av_change_rate_roll_mean_100"]))/2.0)))) - (((data["avg_first_10000"]) * (data["avg_first_10000"]))))) + (2.0))))))) +
            0.0399999991*np.tanh(((data["mean_change_rate_first_10000"]) + (((data["q01_roll_std_1000"]) + (((data["q01_roll_std_10"]) * (((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_1000"]) + (((data["abs_q05"]) * (((((((data["std"]) * 2.0)) * 2.0)) * 2.0)))))))) * ((-1.0*((data["q05_roll_std_100"])))))))))))) +
            0.0399999991*np.tanh(((((data["classic_sta_lta3_mean"]) - (((((((data["av_change_abs_roll_mean_10"]) + (data["q95_roll_mean_10"]))) + (data["av_change_abs_roll_std_100"]))) + (((np.tanh((data["max_to_min"]))) - (((((data["q05"]) - (((data["av_change_rate_roll_std_1000"]) * ((-1.0*((data["max_to_min"])))))))) * 2.0)))))))) * 2.0)) +
            0.0399999991*np.tanh(((((((data["q01_roll_std_10"]) * (((data["Moving_average_700_mean"]) + (data["Hilbert_mean"]))))) + (np.tanh((((((data["q95_roll_std_10"]) * 2.0)) * 2.0)))))) * ((-1.0*((((((((((data["q05_roll_std_100"]) * 2.0)) - (data["avg_last_10000"]))) * 2.0)) - (data["q95_roll_mean_1000"])))))))) +
            0.0399999991*np.tanh(((((data["skew"]) - ((((-1.0*((data["av_change_rate_roll_std_10"])))) * (((data["skew"]) + (data["max_to_min"]))))))) - (((data["q05_roll_std_1000"]) + (((((data["max_to_min"]) + (((((data["kurt"]) + (-1.0))) * 2.0)))) * (data["abs_max_roll_std_100"]))))))) +
            0.0399999991*np.tanh(((((((data["q05_roll_std_1000"]) + (data["exp_Moving_average_3000_mean"]))) * ((-1.0*((((((data["q01_roll_std_1000"]) * (data["q05_roll_std_10"]))) + (((data["max"]) + (data["q01_roll_std_1000"])))))))))) * (((((((data["q05_roll_std_100"]) * 2.0)) * (data["q01_roll_std_100"]))) - (data["q01_roll_std_1000"]))))) +
            0.0399999991*np.tanh(((data["q95_roll_std_1000"]) + (((data["q01_roll_std_100"]) - (((((((data["q05_roll_std_1000"]) + (data["abs_q05"]))) * (data["q05_roll_std_100"]))) * (((((data["q05_roll_std_10"]) + (data["q01_roll_std_10"]))) * (((((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))) + (data["avg_last_10000"]))))))))))) +
            0.0399999991*np.tanh(((((((data["q05_roll_std_1000"]) * (data["q95_roll_mean_10"]))) + (((((((data["q95"]) * (data["q95"]))) * (data["q01_roll_std_10"]))) - (((data["exp_Moving_average_30000_mean"]) * (data["exp_Moving_average_30000_mean"]))))))) * ((((-1.0*((((data["q05_roll_std_1000"]) * (data["q01_roll_std_10"])))))) * 2.0)))) +
            0.0399999991*np.tanh(((((((data["q05_roll_std_10"]) * 2.0)) * 2.0)) * ((-1.0*((((np.tanh((((data["skew"]) + (((data["skew"]) + ((((data["max_to_min"]) + (data["av_change_rate_roll_mean_10"]))/2.0)))))))) + ((((data["q01_roll_std_10"]) + (((data["q95"]) * (data["q95"]))))/2.0))))))))) +
            0.0399999991*np.tanh(((data["min_last_10000"]) - ((((-1.0*((((data["kurt"]) - (data["std_first_10000"])))))) - (((((((data["min"]) * (data["kurt"]))) * 2.0)) + (((data["av_change_abs_roll_std_1000"]) * (((((data["av_change_abs_roll_mean_100"]) + (data["med"]))) + (data["med"]))))))))))) +
            0.0399999991*np.tanh(((((((data["min"]) + (((data["min"]) - (((((data["av_change_abs_roll_mean_10"]) - (((data["av_change_abs_roll_std_1000"]) + (data["av_change_abs_roll_std_10"]))))) + (((((data["avg_last_10000"]) + (data["av_change_abs_roll_std_10"]))) * (data["max_last_50000"]))))))))) * 2.0)) - (np.tanh((data["min_roll_std_100"]))))) +
            0.0399999991*np.tanh(((data["min_roll_std_1000"]) - (((data["q01_roll_std_1000"]) * (((((data["q01_roll_std_10"]) * (((data["q95_roll_mean_100"]) + (np.tanh((data["q05_roll_std_1000"]))))))) * (((((data["q05_roll_std_100"]) * (data["q05_roll_std_100"]))) + ((((data["q95_roll_mean_100"]) + (data["q01_roll_std_100"]))/2.0)))))))))) +
            0.0399921872*np.tanh(((((((data["q01_roll_mean_100"]) * (((((((((data["ave_roll_std_10"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) - (((((data["q01_roll_std_10"]) * (data["q05_roll_std_1000"]))) * (((data["q05_roll_std_1000"]) * (((data["q05_roll_std_1000"]) * (data["q05_roll_std_1000"]))))))))) +
            0.0399999991*np.tanh(((((((data["q95_roll_std_100"]) + (((((data["q95_roll_std_10"]) * (data["avg_first_50000"]))) - (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((((data["q95"]) * (data["q95"]))) + (data["avg_first_50000"]))))))))))))) * 2.0)) * 2.0)) +
            0.0399921872*np.tanh(((data["std_roll_mean_1000"]) * ((((((((-1.0*((((data["med"]) + (((data["med"]) + (data["std_roll_mean_1000"])))))))) - (((data["skew"]) - (((data["q05_roll_std_1000"]) * (((((((data["min_last_10000"]) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)))) +
            0.0399999991*np.tanh(((((((((data["min"]) * (data["kurt"]))) * (data["kurt"]))) * (data["kurt"]))) + (((data["kurt"]) - (np.tanh(((((((-1.0*((((((data["av_change_rate_roll_std_100"]) * (data["q95_roll_mean_1000"]))) - (data["Moving_average_6000_mean"])))))) * 2.0)) * 2.0)))))))) +
            0.0399999991*np.tanh((((((data["q95_roll_std_100"]) + ((((((data["classic_sta_lta4_mean"]) - (data["q05_roll_std_1000"]))) + (data["abs_q05"]))/2.0)))/2.0)) + (((((1.0) - (((((data["q05_roll_std_100"]) - (data["mean_change_rate_last_50000"]))) * (data["q05_roll_std_1000"]))))) + (((data["min_roll_mean_1000"]) - (data["mean_change_rate_last_50000"]))))))) +
            0.0399999991*np.tanh(((data["q95_roll_std_1000"]) - (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((data["q05_roll_std_100"]) * (((((data["med"]) * (data["q95_roll_std_1000"]))) + (((data["q05_roll_std_100"]) * (((((data["q99_roll_mean_1000"]) * (data["q05_roll_std_100"]))) + (data["q05_roll_std_100"]))))))))))))))) +
            0.0399843715*np.tanh(((((((data["skew"]) + (((data["q01_roll_std_1000"]) * (((data["abs_q05"]) - (((data["max_to_min"]) + (data["q95_roll_mean_10"]))))))))) + (((data["mean_change_rate"]) + (data["min_last_10000"]))))) + (((data["min_first_10000"]) * (((data["av_change_rate_roll_std_100"]) + (data["q95_roll_mean_10"]))))))) +
            0.0399999991*np.tanh(((data["q01_roll_std_1000"]) * ((-1.0*((((data["q99_roll_mean_1000"]) + (np.tanh(((((((((((((((data["q01_roll_std_1000"]) + (data["abs_trend"]))/2.0)) - (((data["skew"]) - (((data["Hann_window_mean"]) * 2.0)))))) * (data["q01_roll_std_10"]))) * 2.0)) * 2.0)) * 2.0))))))))))) +
            0.0399999991*np.tanh(((((data["min_roll_mean_1000"]) * 2.0)) + (np.tanh((np.tanh((((((((((data["min_roll_mean_1000"]) + (((((data["av_change_abs_roll_mean_1000"]) + (((((data["min_roll_mean_1000"]) + (data["kurt"]))) * 2.0)))) - (data["mean_change_rate_first_50000"]))))) * 2.0)) * 2.0)) * 2.0)))))))) +
            0.0399921872*np.tanh(((((data["av_change_rate_roll_std_1000"]) * (data["min_roll_std_10"]))) + (((((data["q05_roll_mean_1000"]) + (np.tanh((((data["av_change_rate_roll_mean_10"]) - (((data["avg_first_50000"]) + (data["max_roll_mean_1000"]))))))))) * (((((data["skew"]) * 2.0)) + (((data["skew"]) + (data["q99_roll_mean_1000"]))))))))) +
            0.0399999991*np.tanh(((((data["kurt"]) * (data["max_to_min"]))) + (((np.tanh((((((((((((((((((data["q95_roll_std_100"]) + (data["q05"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) + (np.tanh((data["mean_change_rate_last_10000"]))))))) +
            0.0399999991*np.tanh(((((((((data["q95_roll_mean_10"]) + (((data["q95_roll_mean_10"]) * 2.0)))) - (data["max_to_min"]))) * (((data["q05_roll_mean_100"]) + (((data["MA_400MA_BB_low_mean"]) * ((((((data["q05_roll_mean_100"]) + (((((data["q95_roll_mean_1000"]) * 2.0)) * 2.0)))/2.0)) - (data["max_to_min"]))))))))) * 2.0)) +
            0.0399999991*np.tanh(((data["iqr"]) - (((data["q01_roll_std_100"]) * ((((data["q01_roll_std_100"]) + (((((((data["iqr"]) * (((data["q999"]) + (((data["iqr"]) + (((data["q01_roll_std_10"]) * (data["max_to_min"]))))))))) * (data["q01_roll_std_10"]))) * (data["iqr"]))))/2.0)))))) +
            0.0399999991*np.tanh(((((((data["min_roll_mean_1000"]) + (((data["min_roll_mean_1000"]) + (((((((data["q05"]) - (data["std_last_50000"]))) * (data["min_roll_std_100"]))) + (((data["max_last_10000"]) - (data["kurt"]))))))))) * (((data["std_last_50000"]) * (((data["q95_roll_mean_1000"]) * 2.0)))))) * 2.0)) +
            0.0399921872*np.tanh(((((((((data["q999"]) * (((((data["q01_roll_mean_1000"]) - (data["abs_max_roll_std_100"]))) - (data["std_last_10000"]))))) * 2.0)) + (np.tanh(((((-1.0*((((data["av_change_rate_roll_std_10"]) - (data["std_last_10000"])))))) * (((data["min_roll_std_1000"]) - (data["std_last_10000"]))))))))) * 2.0)) +
            0.0399921872*np.tanh(((((data["av_change_abs_roll_std_1000"]) + (data["q05"]))) * (((((((data["avg_last_10000"]) - (data["classic_sta_lta4_mean"]))) - (data["min_roll_std_10"]))) + (((((((((data["min_roll_std_10"]) - (data["classic_sta_lta4_mean"]))) - (data["q01_roll_mean_1000"]))) + (data["med"]))) * (data["min_roll_std_10"]))))))) +
            0.0399921872*np.tanh(((data["q95_roll_std_1000"]) - (((data["abs_q95"]) - ((((data["mean_change_rate"]) + ((-1.0*(((((((data["av_change_rate_roll_std_100"]) + (((((((data["av_change_rate_roll_mean_1000"]) * 2.0)) - (data["av_change_abs_roll_std_10"]))) * ((((data["mean_change_rate"]) + (data["mean_change_abs"]))/2.0)))))/2.0)) * 2.0))))))/2.0)))))) +
            0.0399765596*np.tanh((((-1.0*((((data["q95_roll_std_1000"]) * (((data["q95_roll_mean_1000"]) * (((data["std_first_50000"]) + (((data["q99_roll_std_10"]) + (data["std_first_50000"])))))))))))) - (((((((data["av_change_abs_roll_mean_100"]) - (((data["max"]) + (data["mean_change_abs"]))))) - (data["q99_roll_mean_10"]))) / 2.0)))) +
            0.0399687439*np.tanh(((np.tanh((((np.tanh((((((((((((np.tanh((((data["mean_change_rate"]) * 2.0)))) - (data["min_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) - (data["av_change_abs_roll_std_10"]))))) + ((((((data["av_change_abs_roll_std_10"]) + (data["q95_roll_std_1000"]))/2.0)) + (data["min_roll_mean_1000"]))))) +
            0.0399843715*np.tanh(((data["av_change_abs_roll_mean_10"]) * (((((data["classic_sta_lta2_mean"]) + ((((((data["exp_Moving_average_30000_mean"]) * (data["classic_sta_lta2_mean"]))) + (((np.tanh((((((((data["av_change_abs_roll_std_100"]) - (((data["q99_roll_mean_1000"]) + (((data["q01_roll_std_10"]) * 2.0)))))) * 2.0)) * 2.0)))) * 2.0)))/2.0)))) * 2.0)))) +
            0.0399921872*np.tanh(((data["std_roll_mean_1000"]) * (((data["trend"]) - (((((data["min_roll_std_100"]) + (((((data["abs_q95"]) * (((data["std_roll_mean_1000"]) - (data["av_change_rate_roll_mean_10"]))))) + (data["q01_roll_std_1000"]))))) + (((data["q05_roll_std_10"]) * (((data["min_roll_std_100"]) - (data["av_change_abs_roll_std_10"]))))))))))) +
            0.0399921872*np.tanh(((((data["q99_roll_std_100"]) * (3.1415927410))) * (((data["q01_roll_mean_1000"]) + (((((((data["q99_roll_std_100"]) * (3.1415927410))) + (data["min_first_50000"]))) * (((((((data["min_first_50000"]) * (((data["abs_mean"]) - (data["min_first_50000"]))))) * 2.0)) * 2.0)))))))) +
            0.0399765596*np.tanh((-1.0*((((data["max_roll_std_100"]) + (np.tanh((((data["max_roll_std_100"]) * (((((data["classic_sta_lta3_mean"]) + (data["min_last_10000"]))) - (((data["q95_roll_std_100"]) - (((((data["classic_sta_lta3_mean"]) + (((data["Moving_average_6000_mean"]) + (data["exp_Moving_average_300_mean"]))))) + (data["classic_sta_lta3_mean"])))))))))))))))) +
            0.0399140455*np.tanh(((data["max_roll_mean_100"]) * (((((((((((((((data["av_change_abs_roll_std_10"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (((data["av_change_rate_roll_mean_1000"]) * (((((-3.0) * (((data["max_first_50000"]) + (data["max_roll_std_1000"]))))) + (data["av_change_abs_roll_std_10"]))))))))) +
            0.0399687439*np.tanh(((data["av_change_abs_roll_mean_1000"]) * ((((((data["min_roll_std_1000"]) + (data["std_last_10000"]))/2.0)) + ((-1.0*((np.tanh((((data["min_roll_std_100"]) + (((data["mean_change_rate_last_50000"]) - ((((data["classic_sta_lta2_mean"]) + ((((((data["classic_sta_lta2_mean"]) + (data["av_change_abs_roll_mean_1000"]))/2.0)) + (data["classic_sta_lta3_mean"]))))/2.0))))))))))))))) +
            0.0399843715*np.tanh(((data["av_change_rate_roll_mean_10"]) - (((data["av_change_rate_roll_mean_100"]) + (((((data["mean_change_rate_first_50000"]) * (data["av_change_rate_roll_mean_10"]))) * ((((((data["mean_change_rate_first_50000"]) * (data["av_change_rate_roll_mean_10"]))) + (((np.tanh((data["kurt"]))) + (((np.tanh((data["q05_roll_mean_1000"]))) + (data["max_to_min"]))))))/2.0)))))))) +
            0.0399921872*np.tanh(((data["av_change_abs_roll_std_100"]) * ((((((data["av_change_abs_roll_std_100"]) + (((((data["abs_mean"]) + (data["q01_roll_std_1000"]))) * 2.0)))/2.0)) * (((((((((data["av_change_abs_roll_mean_100"]) - (data["med"]))) - (((data["classic_sta_lta3_mean"]) * 2.0)))) - (data["trend"]))) - (data["av_change_abs_roll_std_100"]))))))) +
            0.0399921872*np.tanh(((data["mean_change_rate_last_50000"]) * (((data["av_change_abs_roll_std_10"]) + (((data["av_change_abs_roll_std_10"]) + (((data["max_roll_std_100"]) * (((((data["mean_change_rate_last_50000"]) - (((((data["Moving_average_6000_mean"]) * (data["av_change_abs_roll_std_10"]))) * (data["av_change_abs_roll_std_10"]))))) - (((data["q95_roll_mean_1000"]) * (data["mean_change_rate_last_50000"]))))))))))))) +
            0.0399999991*np.tanh(((data["mean_change_rate_last_10000"]) * (((data["ave_roll_std_1000"]) * ((((((((data["min_last_10000"]) + (data["max_to_min_diff"]))/2.0)) - (data["trend"]))) + (((data["min_roll_std_1000"]) + (((((((data["max_to_min_diff"]) + (data["min_last_10000"]))) * 2.0)) + (data["av_change_abs_roll_std_100"]))))))))))) +
            0.0399999991*np.tanh(((((((data["q95"]) * 2.0)) * (((data["av_change_rate_roll_mean_1000"]) * (np.tanh((((((((((((((data["min_first_10000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))))) + (((data["min_roll_mean_10"]) - (((data["classic_sta_lta3_mean"]) * (data["q95"]))))))) +
            0.0399765596*np.tanh(((((-1.0*((((data["mean_change_rate_first_50000"]) * (((data["av_change_abs_roll_std_1000"]) - (data["q01_roll_std_1000"])))))))) + (((data["min_roll_mean_100"]) * (((((((((data["mean_change_rate"]) - (((data["min_roll_std_1000"]) * (((data["av_change_abs_roll_std_1000"]) - (data["mean_change_rate"]))))))) * 2.0)) * 2.0)) * 2.0)))))/2.0)) +
            0.0399843715*np.tanh(((data["q95"]) * (((data["q95"]) * (((data["min_roll_std_10"]) - (((((((data["min_roll_std_10"]) * 2.0)) + ((((data["av_change_abs_roll_std_10"]) + ((((((data["q95"]) * 2.0)) + (data["q95"]))/2.0)))/2.0)))) * (((data["av_change_abs_roll_std_10"]) * (data["classic_sta_lta2_mean"]))))))))))) +
            0.0399687439*np.tanh(((((((data["min_first_50000"]) * 2.0)) * 2.0)) * (((data["av_change_abs_roll_std_10"]) * (((((((data["mean_change_abs"]) + (data["max_last_50000"]))) + ((((((data["mean_change_rate_last_10000"]) + (data["mean_change_abs"]))/2.0)) + (((data["med"]) + (data["med"]))))))) - (-3.0))))))) +
            0.0399999991*np.tanh(((((((data["min_roll_mean_1000"]) * (((((data["max_first_50000"]) * 2.0)) + ((((((np.tanh((data["min_roll_mean_1000"]))) + (data["q95_roll_mean_1000"]))/2.0)) + (((((data["exp_Moving_average_3000_mean"]) * ((((data["max_first_50000"]) + (data["exp_Moving_average_3000_mean"]))/2.0)))) + (data["Moving_average_6000_mean"]))))))))) * 2.0)) * 2.0)) +
            0.0399921872*np.tanh((-1.0*((((((data["abs_trend"]) * ((((10.0)) * (((data["av_change_rate_roll_mean_1000"]) - (((data["min_roll_mean_1000"]) + (np.tanh((((data["mean_change_rate"]) - (data["min_roll_mean_1000"]))))))))))))) * (((data["MA_400MA_BB_high_mean"]) - (data["min_roll_mean_1000"])))))))) +
            0.0399843715*np.tanh(((data["abs_max_roll_mean_100"]) + (((np.tanh((data["max_roll_mean_1000"]))) * (((((data["mean_change_rate_last_10000"]) - ((((((data["min_last_10000"]) + (data["kurt"]))/2.0)) * (data["skew"]))))) - (((data["avg_last_10000"]) + ((((-1.0*((data["q01_roll_mean_1000"])))) * 2.0)))))))))) +
            0.0399999991*np.tanh(((data["q01_roll_std_1000"]) * (((data["q99_roll_mean_100"]) * (((((data["min_roll_std_1000"]) * (((data["min_roll_mean_1000"]) * (data["avg_first_10000"]))))) + (((((data["trend"]) + (((data["q05_roll_std_10"]) * (((data["avg_first_10000"]) * (data["min_roll_mean_1000"]))))))) + (data["av_change_rate_roll_mean_10"]))))))))) +
            0.0399999991*np.tanh((-1.0*((((data["max_first_10000"]) * (((((data["skew"]) * (((((((data["min_roll_std_100"]) * (data["min_last_10000"]))) + (data["q05_roll_std_1000"]))) * (((((data["q05_roll_std_100"]) + (data["q05_roll_std_100"]))) * (data["min_last_10000"]))))))) - (data["trend"])))))))) +
            0.0379058383*np.tanh(np.tanh((((np.tanh((np.tanh((((((data["av_change_abs_roll_mean_10"]) * ((((((((data["min_roll_std_1000"]) + (data["mean_change_rate"]))/2.0)) + ((-1.0*((np.tanh((data["av_change_rate_roll_std_10"])))))))) + (((data["classic_sta_lta2_mean"]) + (data["std_last_10000"]))))))) * 2.0)))))) * 2.0)))) +
            0.0399921872*np.tanh(((((((((data["classic_sta_lta1_mean"]) * 2.0)) * 2.0)) * 2.0)) * (((data["MA_700MA_BB_high_mean"]) * (((data["max_roll_std_10"]) * (((((((data["classic_sta_lta4_mean"]) - (data["av_change_abs_roll_std_1000"]))) - (data["max_roll_std_10"]))) - (((data["max_to_min_diff"]) + (data["av_change_abs_roll_std_1000"]))))))))))) +
            0.0399921872*np.tanh(((((((((data["max_last_50000"]) * 2.0)) + (((data["av_change_abs_roll_mean_100"]) - (np.tanh((-1.0))))))) - (((((((data["min_last_50000"]) + (data["av_change_abs_roll_mean_100"]))) - (data["mean_change_rate_last_10000"]))) * (data["mean_change_rate_first_10000"]))))) * ((((data["mean_change_rate_last_10000"]) + (data["min_last_50000"]))/2.0)))) +
            0.0399921872*np.tanh(((data["mean_change_abs"]) * ((((-1.0*((data["classic_sta_lta3_mean"])))) + (((((((((data["avg_first_10000"]) + ((((data["classic_sta_lta3_mean"]) + (data["avg_last_10000"]))/2.0)))/2.0)) - (((data["max_to_min"]) - (np.tanh((data["min_first_50000"]))))))) + (((data["min_roll_std_10"]) - (data["max_first_10000"]))))/2.0)))))) +
            0.0399687439*np.tanh(((data["std_roll_mean_1000"]) * (((((((data["av_change_abs_roll_std_10"]) * (data["abs_q05"]))) - ((((((data["std_roll_mean_1000"]) * (data["std_roll_mean_1000"]))) + (data["abs_q05"]))/2.0)))) + (((((((data["abs_max_roll_std_1000"]) + (((data["min_last_10000"]) * (data["q95_roll_std_1000"]))))) * 2.0)) * 2.0)))))) +
            0.0396483690*np.tanh(((((data["q05"]) * (((data["avg_first_50000"]) - ((((data["q05_roll_mean_1000"]) + (((data["trend"]) * (data["av_change_rate_roll_std_1000"]))))/2.0)))))) + ((((data["min_last_10000"]) + ((((((data["max_to_min_diff"]) + (((data["max_to_min_diff"]) + (data["av_change_abs_roll_mean_1000"]))))) + (data["classic_sta_lta4_mean"]))/2.0)))/2.0)))) +
            0.0399765596*np.tanh(((((data["q99_roll_std_10"]) * 2.0)) * (((((data["med"]) * (np.tanh((data["q01_roll_mean_10"]))))) - (np.tanh((((((((data["av_change_abs_roll_std_1000"]) * 2.0)) * (((data["med"]) * 2.0)))) - (((((data["Moving_average_6000_mean"]) - (data["classic_sta_lta1_mean"]))) * 2.0)))))))))) +
            0.0399999991*np.tanh(((data["abs_max_roll_mean_100"]) + (((data["min_roll_mean_1000"]) - ((((((data["q95_roll_mean_1000"]) + ((-1.0*((data["abs_q05"])))))/2.0)) * (((data["kurt"]) * (((data["kurt"]) * (((data["q95_roll_std_100"]) + (((((data["kurt"]) / 2.0)) * (data["kurt"]))))))))))))))) +
            0.0399609283*np.tanh((-1.0*((((((data["av_change_rate_roll_mean_100"]) * (data["mean_change_rate_first_10000"]))) * ((((((data["MA_400MA_BB_low_mean"]) * (((((data["max_last_10000"]) + (np.tanh((np.tanh((data["classic_sta_lta4_mean"]))))))) + (np.tanh((data["q05"]))))))) + (((data["classic_sta_lta4_mean"]) + (data["trend"]))))/2.0))))))) +
            0.0399843715*np.tanh(((((((((((data["q05_roll_std_1000"]) - (data["min_roll_std_10"]))) + (((((data["mean"]) * (data["skew"]))) - (((data["min_roll_std_10"]) + (data["max_to_min_diff"]))))))) + (((data["Hann_window_mean"]) * (data["classic_sta_lta1_mean"]))))) * (data["max_first_10000"]))) * (data["av_change_abs_roll_mean_1000"]))) +
            0.0329283066*np.tanh(((((data["std_roll_mean_1000"]) + (((data["std_roll_mean_1000"]) + (((data["abs_trend"]) + (((data["std_roll_mean_1000"]) + (data["Moving_average_3000_mean"]))))))))) * ((((((data["min_roll_std_10"]) + (((data["min_roll_std_10"]) * (data["Moving_average_3000_mean"]))))/2.0)) - (((data["std_roll_mean_1000"]) - (data["min_roll_std_10"]))))))) +
            0.0387106836*np.tanh(np.tanh((((((data["q95_roll_std_1000"]) + (((data["av_change_rate_roll_mean_10"]) * (data["mean_change_rate_last_10000"]))))) + (((((data["q05_roll_mean_10"]) * 2.0)) * (((data["min_roll_std_10"]) * (np.tanh((((((data["mean"]) * 2.0)) + (data["min_roll_std_10"]))))))))))))) +
            0.0399765596*np.tanh((-1.0*((((data["av_change_abs_roll_mean_10"]) * (((data["avg_last_10000"]) * (((((data["mean_change_rate_first_50000"]) - (np.tanh((data["kurt"]))))) - (((((((data["av_change_abs_roll_std_1000"]) + (data["classic_sta_lta1_mean"]))/2.0)) + ((((data["max_to_min"]) + ((((data["q05_roll_std_1000"]) + (data["min_first_50000"]))/2.0)))/2.0)))/2.0))))))))))) +
            0.0399609283*np.tanh((-1.0*((((data["q01_roll_mean_10"]) + (((data["q95"]) - (((data["min_last_10000"]) * (((data["mean_change_abs"]) * ((((((((((data["q95"]) * (data["MA_700MA_BB_high_mean"]))) + (data["min_last_10000"]))/2.0)) + (data["min_first_10000"]))) + (data["min_first_10000"])))))))))))))) +
            0.0399531126*np.tanh((-1.0*(((((data["av_change_rate_roll_mean_1000"]) + ((-1.0*((((data["av_change_rate_roll_std_10"]) + ((((-1.0*(((((((data["avg_last_10000"]) * (((data["q05_roll_std_100"]) - (((data["av_change_rate_roll_std_10"]) * (data["min_last_10000"]))))))) + (((data["av_change_rate_roll_std_10"]) * (data["av_change_abs_roll_std_10"]))))/2.0))))) / 2.0))))))))/2.0))))) +
            0.0398984179*np.tanh(((((data["q95_roll_mean_10"]) * (np.tanh((((data["mean_change_rate_first_50000"]) * (((data["mean_change_rate_first_10000"]) - (((((((data["av_change_abs_roll_std_1000"]) + (data["std"]))) + ((-1.0*(((((data["max_last_10000"]) + (((data["classic_sta_lta2_mean"]) * (data["av_change_abs_roll_std_10"]))))/2.0))))))) * 2.0)))))))))) * 2.0)) +
            0.0399843715*np.tanh(((((((data["av_change_abs_roll_mean_100"]) * ((-1.0*((data["mean_change_rate_first_10000"])))))) - (((data["min_roll_std_1000"]) * (data["q01_roll_mean_10"]))))) - (((data["q01_roll_std_10"]) * (((((data["av_change_abs_roll_mean_100"]) * (data["MA_700MA_std_mean"]))) * (((data["av_change_abs_roll_mean_100"]) - (np.tanh((data["mean_change_rate_first_50000"]))))))))))) +
            0.0398984179*np.tanh(((((data["max_first_10000"]) * (((np.tanh((((data["classic_sta_lta1_mean"]) * 2.0)))) * ((-1.0*((np.tanh((((((((((data["avg_last_50000"]) * (((((data["min_first_10000"]) - (data["min_roll_std_10"]))) - (data["avg_last_50000"]))))) * 2.0)) * 2.0)) * 2.0))))))))))) * 2.0)) +
            0.0399687439*np.tanh(((data["min_first_10000"]) * (((((((((((((data["q99_roll_mean_10"]) + (data["abs_max_roll_std_10"]))/2.0)) * (data["av_change_abs_roll_std_1000"]))) + (data["abs_max"]))/2.0)) + (((data["std_roll_mean_1000"]) + (((((data["q01_roll_mean_1000"]) + (data["q99_roll_mean_10"]))) * (((data["min_first_50000"]) * 2.0)))))))) * 2.0)))) +
            0.0399296731*np.tanh(((((((data["max_to_min_diff"]) * (((data["min_first_50000"]) + ((((((data["kurt"]) + (((data["av_change_abs_roll_std_10"]) + (data["std_first_10000"]))))) + ((((((data["kurt"]) - (data["av_change_rate_roll_mean_1000"]))) + (data["min_roll_std_10"]))/2.0)))/2.0)))))) * 2.0)) * 2.0)) +
            0.0398906022*np.tanh(((data["q95_roll_mean_100"]) * ((-1.0*(((((((data["mean_change_rate_last_10000"]) * (data["Moving_average_6000_mean"]))) + ((((((data["count_big"]) + (data["mean_change_rate_last_10000"]))) + (((data["mean_change_rate_last_10000"]) + (((data["mean_change_rate_last_10000"]) + ((((np.tanh((data["mean_change_rate_last_10000"]))) + (data["min_first_50000"]))/2.0)))))))/2.0)))/2.0))))))) +
            0.0399843715*np.tanh(((((data["classic_sta_lta1_mean"]) * (((data["av_change_rate_roll_std_100"]) + (((np.tanh((((np.tanh((((data["av_change_rate_roll_std_10"]) * (data["av_change_abs_roll_std_1000"]))))) + (0.3183098733))))) - (((data["av_change_rate_roll_std_10"]) * (data["Hann_window_mean"]))))))))) * (((data["mean_change_rate_last_10000"]) * (data["av_change_abs_roll_std_1000"]))))) +
            0.0399999991*np.tanh(((((data["av_change_rate_roll_mean_100"]) * (((data["av_change_rate_roll_mean_100"]) * ((((((data["classic_sta_lta3_mean"]) + (data["ave_roll_mean_1000"]))) + (data["av_change_rate_roll_mean_100"]))/2.0)))))) * (((data["ave_roll_mean_1000"]) * (((data["classic_sta_lta3_mean"]) * (((((((data["ave_roll_mean_1000"]) + (data["classic_sta_lta4_mean"]))/2.0)) + (data["classic_sta_lta4_mean"]))/2.0)))))))) +
            0.0399453007*np.tanh((((((((data["count_big"]) - (data["min_first_50000"]))) * ((((((((data["abs_max_roll_mean_10"]) + (data["classic_sta_lta4_mean"]))/2.0)) + (data["q01_roll_mean_1000"]))) + (data["q01_roll_mean_1000"]))))) + ((((((data["abs_max_roll_mean_10"]) - (data["min_first_50000"]))) + (((data["min_roll_std_10"]) - (data["min_first_50000"]))))/2.0)))/2.0)) +
            0.0355147459*np.tanh(((((((data["min_last_10000"]) * (data["classic_sta_lta3_mean"]))) * ((-1.0*((data["min_first_50000"])))))) - ((((((((data["q01_roll_std_1000"]) * (data["mean_change_abs"]))) * (data["classic_sta_lta2_mean"]))) + (np.tanh((((data["classic_sta_lta2_mean"]) * (data["trend"]))))))/2.0)))) +
            0.0355928876*np.tanh((((-1.0*((data["min"])))) * (((((((data["mean_change_rate_last_10000"]) - (data["av_change_abs_roll_std_1000"]))) * (data["Moving_average_1500_mean"]))) - (((((((((data["av_change_abs_roll_std_1000"]) * (data["mean_change_rate_first_50000"]))) - (((data["mean_change_rate_last_10000"]) * 2.0)))) - (data["q95_roll_std_10"]))) - (data["mean_change_rate_last_10000"]))))))) +
            0.0399999991*np.tanh(((((-1.0*(((((((data["std_last_10000"]) * (data["mean_change_rate_last_10000"]))) + (((data["max_roll_mean_10"]) * (((((data["min_roll_mean_1000"]) + (data["av_change_rate_roll_std_100"]))) * (data["q001"]))))))/2.0))))) + (((data["min_roll_mean_1000"]) + (((data["min_roll_mean_1000"]) + (data["abs_trend"]))))))/2.0)) +
            0.0399921872*np.tanh(((data["av_change_abs_roll_std_100"]) * (((data["std_roll_mean_1000"]) + ((((data["trend"]) + ((((((((((data["trend"]) * (data["mean_change_abs"]))) + (data["mean_change_abs"]))) + (((data["av_change_abs_roll_std_100"]) / 2.0)))/2.0)) + ((-1.0*((((data["med"]) * (data["mean_change_abs"])))))))))/2.0)))))) +
            0.0399999991*np.tanh((((((data["q95_roll_std_100"]) - (data["q95"]))) + ((((data["min_last_10000"]) + (((np.tanh((((((data["std_roll_mean_1000"]) * 2.0)) * 2.0)))) + (np.tanh((((((((((data["mean_change_rate_last_10000"]) + (data["q95_roll_std_100"]))) * 2.0)) + (data["classic_sta_lta3_mean"]))) * 2.0)))))))/2.0)))/2.0)) +
            0.0331861675*np.tanh(((((data["std_first_10000"]) * (((data["std_first_10000"]) * (data["av_change_abs_roll_std_1000"]))))) + (((data["mean_change_rate_last_50000"]) * (((np.tanh((((data["max"]) * 2.0)))) * (((((((data["mean"]) - (data["max_first_10000"]))) - (data["max_first_10000"]))) - (data["max_to_min"]))))))))) +
            0.0399999991*np.tanh(((((((data["min_first_10000"]) + (((data["max_to_min"]) * (data["mean_change_rate"]))))) * (((((data["max_roll_mean_10"]) * 2.0)) * (((data["av_change_rate_roll_mean_1000"]) - (((((((data["av_change_abs_roll_mean_1000"]) + (data["mean_change_rate"]))/2.0)) + ((-1.0*((data["ave10"])))))/2.0)))))))) * 2.0)) +
            0.0339441299*np.tanh(((((data["max_to_min"]) * (data["min_roll_mean_100"]))) + (((((((data["avg_first_50000"]) * (data["mean_change_rate_last_50000"]))) + (((data["mean_change_rate_last_50000"]) + (((data["abs_max_roll_mean_10"]) * (data["abs_max_roll_mean_10"]))))))) * (((data["min_roll_std_10"]) + (((data["abs_max_roll_mean_10"]) * (data["avg_first_50000"]))))))))) +
            0.0395545997*np.tanh(((((((((data["min_roll_std_100"]) * (((data["avg_last_10000"]) * (data["kurt"]))))) + ((((((data["std_first_10000"]) + (data["kurt"]))/2.0)) - (data["min_roll_std_100"]))))/2.0)) + (((data["avg_last_10000"]) * (((data["std_first_10000"]) - ((((data["kurt"]) + (data["min_roll_std_100"]))/2.0)))))))/2.0)) +
            0.0399843715*np.tanh((((((((-1.0*((((data["classic_sta_lta3_mean"]) * (data["q99_roll_std_1000"])))))) - ((((((data["max_to_min_diff"]) + (((data["kurt"]) * (data["std_roll_mean_1000"]))))/2.0)) + (((data["min_roll_std_10"]) - (data["MA_1000MA_std_mean"]))))))) * 2.0)) * (((data["kurt"]) * (data["std_roll_mean_1000"]))))) +
            0.0399999991*np.tanh((((((data["classic_sta_lta3_mean"]) * ((((((np.tanh((np.tanh((data["max_last_10000"]))))) + (data["skew"]))) + (((data["min_roll_mean_100"]) * 2.0)))/2.0)))) + (((data["skew"]) * ((-1.0*((((((np.tanh((data["max_last_10000"]))) + (data["skew"]))) / 2.0))))))))/2.0)) +
            0.0399296731*np.tanh(((data["min_roll_mean_100"]) + ((((data["max_roll_mean_10"]) + (((((((data["min_last_50000"]) * 2.0)) * 2.0)) * (((data["min_roll_mean_100"]) + (((data["max_roll_mean_10"]) * (((data["max_roll_mean_10"]) * ((((((data["max_roll_mean_10"]) + (data["min_roll_mean_100"]))/2.0)) - (data["min_roll_std_10"]))))))))))))/2.0)))) +
            0.0399999991*np.tanh(((((np.tanh((data["av_change_rate_roll_std_1000"]))) + (((data["mean_change_rate"]) - (np.tanh((data["exp_Moving_average_30000_mean"]))))))) * (((data["trend"]) * ((-1.0*(((((np.tanh((data["exp_Moving_average_300_mean"]))) + (((data["av_change_rate_roll_std_1000"]) - (((data["min_roll_std_1000"]) * (data["av_change_rate_roll_std_1000"]))))))/2.0))))))))) +
            0.0399921872*np.tanh(((((((data["max"]) * (((data["max_first_50000"]) * (((((((data["q95_roll_mean_10"]) * (((((data["max"]) * (((data["mean_change_rate_last_50000"]) - (data["av_change_abs_roll_std_1000"]))))) - (data["av_change_abs_roll_mean_10"]))))) * 2.0)) - (data["av_change_abs_roll_mean_10"]))))))) * 2.0)) * 2.0)) +
            0.0399609283*np.tanh(np.tanh(((((((data["med"]) * ((((data["av_change_rate_roll_std_100"]) + (((data["av_change_abs_roll_mean_100"]) * (data["av_change_rate_roll_std_100"]))))/2.0)))) + (((data["mean_change_rate_first_50000"]) * (((((-1.0*((((data["max_to_min_diff"]) + (data["min_roll_std_10"])))))) + (((data["av_change_abs_roll_mean_100"]) * (data["min_last_10000"]))))/2.0)))))/2.0)))) +
            0.0399921872*np.tanh(((data["av_change_abs_roll_std_100"]) * ((-1.0*((((((((((((data["av_change_abs_roll_std_100"]) - (data["q95_roll_mean_100"]))) * (((np.tanh((((((data["mean"]) * (data["av_change_rate_roll_mean_100"]))) * 2.0)))) * 2.0)))) + (data["mean"]))/2.0)) + (((data["classic_sta_lta3_mean"]) - (data["skew"]))))/2.0))))))) +
            0.0375073254*np.tanh(((data["av_change_abs_roll_std_1000"]) * ((((((data["classic_sta_lta2_mean"]) + ((-1.0*((((data["min_roll_std_1000"]) + (data["min_roll_std_1000"])))))))) + (((data["min_roll_std_10"]) * (((((data["av_change_abs_roll_mean_10"]) * 2.0)) + (((((data["min_roll_std_100"]) + (data["min_roll_std_100"]))) + (data["classic_sta_lta2_mean"]))))))))/2.0)))) +
            0.0399921872*np.tanh(((data["max_to_min"]) * (((((data["av_change_rate_roll_std_100"]) - (((np.tanh((data["av_change_abs_roll_mean_100"]))) + (((np.tanh((data["mean_change_rate_first_50000"]))) + ((((data["av_change_abs_roll_std_1000"]) + (((data["min_roll_std_10"]) + (data["min_roll_std_10"]))))/2.0)))))))) * (((data["av_change_abs_roll_mean_1000"]) * (data["min_last_10000"]))))))) +
            0.0352881439*np.tanh(((((data["std_first_50000"]) * (np.tanh((((data["q05_roll_mean_1000"]) * (((((data["classic_sta_lta4_mean"]) - (((((data["av_change_abs_roll_mean_10"]) * 2.0)) - ((((((-1.0*((data["av_change_rate_roll_std_10"])))) - (data["av_change_abs_roll_mean_10"]))) - (data["q05_roll_mean_1000"]))))))) - (data["av_change_rate_roll_std_10"]))))))))) * 2.0)) +
            0.0390310585*np.tanh(((np.tanh((((((((((data["min_roll_mean_1000"]) + (((data["MA_400MA_BB_high_mean"]) - (((data["q95_roll_mean_10"]) * (((((data["av_change_rate_roll_mean_10"]) * (data["iqr"]))) + (((data["av_change_rate_roll_std_10"]) + (data["min_last_10000"]))))))))))) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) +
            0.0393045507*np.tanh(((np.tanh((((((((data["q95_roll_mean_1000"]) + (((data["classic_sta_lta3_mean"]) + (data["q95_roll_mean_1000"]))))) + (((data["classic_sta_lta3_mean"]) + (((data["abs_q05"]) + (((data["classic_sta_lta3_mean"]) + (data["classic_sta_lta1_mean"]))))))))) * (((data["avg_first_10000"]) - (data["mean_change_abs"]))))))) / 2.0)) +
            0.0396796241*np.tanh(((np.tanh(((((((data["min_last_50000"]) + (data["classic_sta_lta4_mean"]))/2.0)) + (np.tanh((((((((((data["av_change_abs_roll_std_10"]) + (((np.tanh((np.tanh((((((data["q95_roll_mean_1000"]) + (data["kurt"]))) * 2.0)))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))))))) / 2.0)) +
            0.0399609283*np.tanh(((data["min_first_10000"]) * (np.tanh((((((((((data["mean_change_rate_last_10000"]) - (((data["skew"]) * (data["min_first_10000"]))))) + (((((data["mean_change_rate_last_10000"]) + (data["q05_roll_mean_10"]))) + (((((data["mean_change_rate_last_10000"]) + (data["min_first_10000"]))) * 2.0)))))) * 2.0)) * 2.0)))))) +
            0.0398280881*np.tanh(((((((np.tanh((((((((data["min_roll_mean_100"]) * 2.0)) * 2.0)) - (data["std_first_10000"]))))) + (((np.tanh((data["min_last_50000"]))) * (data["av_change_abs_roll_std_10"]))))/2.0)) + (((data["std_first_10000"]) * ((((((data["min_last_50000"]) * (data["av_change_abs_roll_std_10"]))) + (data["av_change_abs_roll_std_10"]))/2.0)))))/2.0)) +
            0.0399531126*np.tanh(((((((data["av_change_abs_roll_mean_1000"]) * (data["std_last_10000"]))) - (((((data["min_roll_std_10"]) + (np.tanh((((data["avg_last_50000"]) * (data["min_roll_std_10"]))))))) + (np.tanh((((data["av_change_abs_roll_mean_1000"]) + (((data["abs_q05"]) + (data["av_change_rate_roll_mean_1000"]))))))))))) * (data["av_change_abs_roll_std_1000"]))) +
            0.0399296731*np.tanh(np.tanh((np.tanh((((((((((((data["q95_roll_std_100"]) + ((-1.0*((data["q01_roll_std_10"])))))) * 2.0)) * 2.0)) + (data["q95_roll_std_100"]))) * ((((data["q99_roll_mean_1000"]) + ((((((np.tanh((data["q05_roll_std_10"]))) + (data["mean_change_abs"]))/2.0)) + (data["classic_sta_lta1_mean"]))))/2.0)))))))) +
            0.0383121707*np.tanh((((np.tanh(((((((-1.0*((((((data["avg_last_10000"]) + (data["mean_change_abs"]))) + (((data["max_last_10000"]) * (data["max_last_10000"])))))))) + (data["abs_q05"]))) * (data["min_roll_std_100"]))))) + (((((data["avg_last_10000"]) + (data["q01_roll_std_100"]))) * (data["av_change_abs_roll_std_1000"]))))/2.0)) +
            0.0399687439*np.tanh(((((data["mean_change_abs"]) * (((data["min_roll_std_100"]) - (((((((((data["q01_roll_std_1000"]) + (data["mean_change_abs"]))/2.0)) + (((((((((data["abs_trend"]) + (data["av_change_abs_roll_std_1000"]))/2.0)) + (data["av_change_abs_roll_std_1000"]))/2.0)) * (data["min_roll_std_100"]))))/2.0)) * (data["min_roll_std_100"]))))))) * (data["av_change_abs_roll_std_1000"]))) +
            0.0399531126*np.tanh(((np.tanh((np.tanh((((((((((((((data["abs_max_roll_mean_10"]) * (((data["std_first_50000"]) - (((((data["q05_roll_mean_10"]) + (data["min_roll_std_100"]))) + (data["q05_roll_mean_10"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) / 2.0)) +
            0.0399765596*np.tanh(((((data["mean_change_rate_last_50000"]) * ((((data["avg_first_10000"]) + ((-1.0*((((data["ave10"]) - (np.tanh((np.tanh(((((-1.0*((data["mean_change_rate_last_50000"])))) - (1.0)))))))))))))/2.0)))) * ((((2.0) + ((-1.0*((data["mean_change_rate_last_50000"])))))/2.0)))) +
            0.0399453007*np.tanh(((((-1.0*((((data["av_change_abs_roll_std_10"]) * (((((data["ave_roll_mean_100"]) * (((data["av_change_abs_roll_std_10"]) + (data["mean_change_rate_first_50000"]))))) + (data["mean_change_rate_first_50000"])))))))) + (np.tanh(((((((data["classic_sta_lta3_mean"]) + (data["mean_change_rate_first_50000"]))/2.0)) * (data["av_change_abs_roll_mean_100"]))))))/2.0)) +
            0.0399843715*np.tanh(((data["std_first_10000"]) * (((data["classic_sta_lta3_mean"]) * (((np.tanh((((np.tanh((((data["mean_change_rate_first_50000"]) - (((data["q01_roll_std_1000"]) * (data["mean_change_rate_first_50000"]))))))) * 2.0)))) + (np.tanh((((((data["q05_roll_std_1000"]) + (((data["kurt"]) * 2.0)))) * 2.0)))))))))) +
            0.0398984179*np.tanh(((((data["av_change_rate_roll_std_10"]) * (data["av_change_abs_roll_std_1000"]))) * (((data["q99_roll_mean_10"]) + (((data["min"]) + ((((data["sum"]) + ((((data["q95_roll_std_100"]) + ((((((data["av_change_abs_roll_std_1000"]) + (data["min_roll_std_1000"]))/2.0)) + (((3.0) - (data["av_change_rate_roll_std_10"]))))))/2.0)))/2.0)))))))) +
            0.0399374850*np.tanh(((((data["av_change_abs_roll_std_100"]) + (((((data["avg_first_10000"]) + (((data["av_change_abs_roll_std_100"]) * (((data["avg_last_10000"]) + (np.tanh((((data["max_last_10000"]) + (data["av_change_abs_roll_std_100"]))))))))))) + (data["std_last_10000"]))))) * (((data["q01_roll_mean_100"]) * (np.tanh((data["max_last_10000"]))))))) +
            0.0399999991*np.tanh(((data["min_last_10000"]) * (((data["mean_change_rate_first_10000"]) * (np.tanh((((((((((((data["av_change_abs_roll_mean_1000"]) + (((((data["av_change_rate_roll_mean_1000"]) + (((data["Moving_average_1500_mean"]) * (((data["av_change_abs_roll_mean_1000"]) + (data["max_last_50000"]))))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))))) +
            0.0398906022*np.tanh(np.tanh((np.tanh(((((((data["abs_max_roll_mean_10"]) + (((data["min_roll_std_10"]) * (((data["std_roll_mean_1000"]) + (((data["max_roll_mean_10"]) + (data["min_last_10000"]))))))))/2.0)) + (((data["abs_trend"]) * (((((data["skew"]) + (data["abs_max_roll_mean_10"]))) + (data["mean_change_abs"]))))))))))) +
            0.0383356102*np.tanh(((((((data["trend"]) * (((data["min_first_10000"]) * (((np.tanh((((((data["mean_change_rate_last_10000"]) - (data["av_change_abs_roll_mean_100"]))) - (data["av_change_abs_roll_mean_100"]))))) + ((((((data["av_change_abs_roll_std_100"]) - (((1.0) / 2.0)))) + (data["av_change_abs_roll_mean_100"]))/2.0)))))))) * 2.0)) * 2.0)) +
            0.0399296731*np.tanh(((np.tanh((((((np.tanh((data["max_first_10000"]))) - (data["av_change_abs_roll_std_1000"]))) + (data["max_first_10000"]))))) * (((((data["ave_roll_std_10"]) * (data["min_roll_std_100"]))) * (((((data["avg_first_50000"]) - ((-1.0*((np.tanh((data["max_first_10000"])))))))) - (data["av_change_abs_roll_mean_100"]))))))) +
            0.0309279151*np.tanh(np.tanh(((((data["q95_roll_std_1000"]) + (((data["min_first_10000"]) + (((((data["av_change_abs_roll_mean_1000"]) * (((((data["av_change_abs_roll_mean_100"]) - (data["av_change_abs_roll_std_1000"]))) - (data["mean_change_abs"]))))) - ((((data["av_change_abs_roll_std_1000"]) + ((((data["MA_700MA_std_mean"]) + (data["av_change_abs_roll_mean_100"]))/2.0)))/2.0)))))))/2.0)))) +
            0.0398671627*np.tanh(((np.tanh((((data["avg_last_50000"]) * (((((data["std_roll_mean_1000"]) - (data["avg_last_50000"]))) - (((((((((np.tanh((data["classic_sta_lta1_mean"]))) * 2.0)) - (data["std_roll_mean_1000"]))) - (((data["classic_sta_lta1_mean"]) * (data["std_roll_mean_1000"]))))) * 2.0)))))))) * (data["std_roll_mean_1000"]))) +
            0.0399687439*np.tanh((((((np.tanh((((((((data["mean_change_rate_first_10000"]) - (data["avg_last_10000"]))) * (data["max_last_10000"]))) + (data["mean_change_rate_last_10000"]))))) + (np.tanh((((((((((data["mean_change_rate_first_10000"]) - (((data["mean_change_rate_first_10000"]) * (data["mean_change_rate_first_10000"]))))) * 2.0)) * 2.0)) * 2.0)))))/2.0)) / 2.0)) +
            0.0398984179*np.tanh(((data["min_roll_std_10"]) * (((((((((data["max_last_50000"]) * (((((data["max_last_50000"]) * (((data["min_roll_std_10"]) * (((data["av_change_abs_roll_mean_10"]) * 2.0)))))) - (data["av_change_abs_roll_mean_10"]))))) - (((data["min_roll_std_10"]) * (data["av_change_abs_roll_mean_10"]))))) - (data["av_change_abs_roll_mean_10"]))) / 2.0)))) +
            0.0399843715*np.tanh((((((data["std_first_10000"]) * (data["av_change_abs_roll_mean_10"]))) + (((((((((((data["MA_400MA_BB_high_mean"]) * (data["avg_first_50000"]))) + (data["max_first_50000"]))) + ((((((data["std_first_10000"]) + (data["max_first_50000"]))) + (data["av_change_abs_roll_mean_10"]))/2.0)))/2.0)) + ((-1.0*((data["av_change_abs_roll_std_100"])))))/2.0)))/2.0)) +
            0.0333658904*np.tanh((((np.tanh((((((((((((((data["abs_q05"]) * (((data["mean_change_rate_first_10000"]) * (((data["q01_roll_std_1000"]) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) + (data["av_change_abs_roll_mean_10"]))) * 2.0)))) + (((((data["mean_change_rate_first_10000"]) * (data["iqr"]))) * (data["min_roll_std_1000"]))))/2.0)) +
            0.0399921872*np.tanh(((data["max_roll_mean_100"]) * (((((data["kurt"]) * ((((data["classic_sta_lta4_mean"]) + (data["min_first_10000"]))/2.0)))) - ((((((((((data["classic_sta_lta4_mean"]) - (data["max_roll_mean_100"]))) + ((((data["mean_change_rate_first_10000"]) + ((3.0)))/2.0)))/2.0)) * (data["classic_sta_lta4_mean"]))) * (data["mean_change_rate_first_10000"]))))))) +
            0.0336081274*np.tanh(((np.tanh((np.tanh((((((data["q95_roll_std_100"]) * (((data["min_roll_std_100"]) - (((((data["MA_400MA_BB_high_mean"]) + (((data["av_change_rate_roll_std_10"]) + (((data["av_change_rate_roll_std_10"]) - (data["classic_sta_lta4_mean"]))))))) + (((data["av_change_rate_roll_std_10"]) - (data["classic_sta_lta4_mean"]))))))))) * 2.0)))))) / 2.0)) +
            0.0399140455*np.tanh((-1.0*(((((((data["av_change_abs_roll_std_10"]) * (np.tanh((data["classic_sta_lta2_mean"]))))) + (((data["mean_change_abs"]) * (np.tanh((((data["mean_change_rate_last_10000"]) - (((data["classic_sta_lta2_mean"]) * ((((((data["classic_sta_lta2_mean"]) * (np.tanh((data["classic_sta_lta2_mean"]))))) + (data["classic_sta_lta2_mean"]))/2.0)))))))))))/2.0))))) +
            0.0399999991*np.tanh(((((np.tanh((((((((((((((((data["q95_roll_std_100"]) - (data["q01_roll_mean_1000"]))) + (np.tanh((((((data["av_change_abs_roll_mean_10"]) * (((data["q01_roll_std_1000"]) + (data["abs_q05"]))))) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0)) +
            0.0353428386*np.tanh(((((((data["min_roll_std_100"]) * ((((((data["Moving_average_6000_mean"]) + ((((data["abs_max_roll_mean_100"]) + (data["min_roll_mean_10"]))/2.0)))/2.0)) * (data["std_roll_mean_10"]))))) - (np.tanh((((data["q05_roll_std_1000"]) - (((((((data["av_change_abs_roll_mean_100"]) + (data["q05_roll_std_1000"]))/2.0)) + (data["q95_roll_std_1000"]))/2.0)))))))) / 2.0)) +
            0.0399609283*np.tanh(((np.tanh((((((((((((((((((((data["av_change_abs_roll_std_1000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + ((((data["ave_roll_mean_1000"]) + (data["av_change_rate_roll_std_1000"]))/2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * (((data["med"]) - (np.tanh((data["exp_Moving_average_300_mean"]))))))) +
            0.0399531126*np.tanh((-1.0*(((((((np.tanh((((data["q01_roll_mean_1000"]) * (data["mean_change_rate_first_50000"]))))) + (data["mean_change_rate_first_50000"]))/2.0)) * ((((((((data["av_change_abs_roll_std_10"]) * (data["mean_change_abs"]))) / 2.0)) + (data["av_change_abs_roll_std_10"]))/2.0))))))) +
            0.0398671627*np.tanh(((((np.tanh((((((((((((((((((((data["av_change_abs_roll_std_100"]) + (((((data["av_change_abs_roll_std_100"]) + (((data["av_change_abs_roll_std_100"]) * 2.0)))) * 2.0)))) * 2.0)) + (data["min_roll_mean_1000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0)) +
            0.0399374850*np.tanh(((np.tanh((np.tanh((((data["av_change_abs_roll_mean_1000"]) * (((((((((((((((data["av_change_abs_roll_std_100"]) + (((np.tanh((((((data["q99_roll_std_100"]) - (data["av_change_abs_roll_std_100"]))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))))) / 2.0)) +
            0.0399687439*np.tanh(((((((data["av_change_abs_roll_std_10"]) * (((data["trend"]) + (((((((data["trend"]) * (((data["trend"]) / 2.0)))) - (data["mean_change_rate"]))) * (((data["trend"]) * (data["avg_last_10000"]))))))))) / 2.0)) * (data["mean_change_rate_first_10000"]))) +
            0.0363195948*np.tanh(np.tanh((((data["q05_roll_std_1000"]) * (((((data["mean_change_abs"]) * (((data["q05_roll_std_1000"]) * (((data["max_to_min_diff"]) * 2.0)))))) - (((((((data["av_change_abs_roll_mean_1000"]) + (((data["mean_change_abs"]) + (data["min_roll_std_10"]))))/2.0)) + (data["max_first_50000"]))/2.0)))))))) +
            0.0398671627*np.tanh(((np.tanh((np.tanh((((data["Moving_average_6000_mean"]) * (((data["q95_roll_std_100"]) * ((((((((np.tanh((((((((((((data["q001"]) - (data["std_first_50000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) + (data["q99_roll_mean_1000"]))/2.0)) * 2.0)))))))))) / 2.0)) +
            0.0354444198*np.tanh((((((((data["min_roll_std_1000"]) * (((data["av_change_abs_roll_std_10"]) + (((data["min_roll_std_100"]) + (data["av_change_abs_roll_mean_100"]))))))) + (np.tanh((((((((((data["Hann_window_mean"]) + (((data["avg_first_50000"]) + (((data["mean_change_rate"]) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)))))/2.0)) / 2.0)) +
            0.0398984179*np.tanh(((np.tanh(((-1.0*((np.tanh((((((((data["q01_roll_std_10"]) * (((data["q05_roll_std_1000"]) * (data["min_roll_std_1000"]))))) * (((data["q95_roll_mean_10"]) * (((((((data["q95_roll_mean_10"]) * (data["q05_roll_std_1000"]))) * 2.0)) * 2.0)))))) + (data["av_change_abs_roll_std_100"])))))))))) / 2.0)) +
            0.0377651863*np.tanh(np.tanh((((data["q001"]) * (((((data["q95"]) + (((data["mean_change_rate_first_50000"]) + ((((((data["av_change_rate_roll_mean_100"]) * (data["mean_change_rate_first_50000"]))) + (((((data["mean_change_rate_first_50000"]) + (data["av_change_rate_roll_mean_100"]))) * (data["min_last_50000"]))))/2.0)))))) * (((data["classic_sta_lta4_mean"]) * 2.0)))))))) +
            0.0399531126*np.tanh(((np.tanh((((data["mean_change_abs"]) * (((data["max_last_10000"]) * (((data["q01_roll_std_100"]) + (((((((((data["mean_change_rate_last_10000"]) - (data["kurt"]))) * 2.0)) + (((data["max_last_10000"]) + (((data["med"]) - (data["max_roll_std_1000"]))))))) * 2.0)))))))))) / 2.0)) +
            0.0399687439*np.tanh(((np.tanh((((((data["mean_change_rate_first_50000"]) * ((((((-1.0*((data["mean_change_rate_first_50000"])))) + (((data["min_first_10000"]) + (data["mean_change_abs"]))))) + ((-1.0*((data["std_roll_mean_1000"])))))))) + ((-1.0*(((((np.tanh((data["mean_change_rate_first_50000"]))) + (-1.0))/2.0))))))))) / 2.0)) +
            0.0399218611*np.tanh((((((((data["av_change_rate_roll_std_10"]) + (data["abs_max_roll_mean_10"]))/2.0)) * (((((data["max_first_50000"]) * 2.0)) - (((((-1.0*((((data["avg_last_50000"]) - (data["av_change_abs_roll_mean_100"])))))) + (data["mean_change_rate_first_10000"]))/2.0)))))) * ((((data["min_roll_std_100"]) + (data["min_first_10000"]))/2.0)))) +
            0.0392185971*np.tanh(((data["min_roll_std_10"]) * (((((data["max_roll_mean_1000"]) * ((-1.0*((((data["ave_roll_std_100"]) + (((((data["av_change_rate_roll_std_10"]) * (((((data["max_to_min"]) * (data["iqr"]))) + ((((data["skew"]) + (data["MA_700MA_std_mean"]))/2.0)))))) + (data["skew"])))))))))) * 2.0)))) +
            0.0364133604*np.tanh((((((np.tanh((((((((((((((((((0.3183098733) + (data["avg_last_50000"]))) * 2.0)) - (data["av_change_rate_roll_mean_100"]))) * 2.0)) * 2.0)) * 2.0)) - (data["mean_change_rate_last_10000"]))) * 2.0)))) + (0.3183098733))/2.0)) * (np.tanh((((data["min_last_10000"]) * 2.0)))))) +
            0.0361398719*np.tanh(((data["min_roll_std_10"]) * (np.tanh((((((data["max_last_10000"]) * 2.0)) * (((((data["mean_change_rate_first_10000"]) * (data["q05_roll_std_100"]))) + (((((((data["q05_roll_std_100"]) + (data["av_change_abs_roll_mean_100"]))) - (data["min_first_10000"]))) - (data["min_first_10000"]))))))))))) +
            0.0399531126*np.tanh(((((data["av_change_rate_roll_std_1000"]) * (np.tanh(((((data["min_last_50000"]) + (((data["MA_700MA_BB_high_mean"]) - (((data["av_change_abs_roll_std_100"]) * (((data["min_roll_std_10"]) - (((((((data["q95"]) - (data["min_roll_std_10"]))) - (data["av_change_abs_roll_std_10"]))) - (data["av_change_abs_roll_std_1000"]))))))))))/2.0)))))) / 2.0)) +
            0.0396874361*np.tanh((((((data["std_roll_mean_1000"]) * (data["q05_roll_mean_100"]))) + (((((((((data["abs_max_roll_mean_100"]) + (((((((((data["mean_change_rate_first_50000"]) * (data["std_roll_mean_1000"]))) + (data["avg_first_10000"]))/2.0)) + (data["min_roll_std_1000"]))/2.0)))/2.0)) * (data["q01_roll_std_10"]))) + (((data["mean_change_rate_first_50000"]) * (data["abs_max_roll_mean_100"]))))/2.0)))/2.0)) +
            0.0399765596*np.tanh(((np.tanh((((((data["av_change_abs_roll_std_10"]) + ((((((((0.72163242101669312)) + (data["q001"]))) + (data["q001"]))) * 2.0)))) * 2.0)))) * (((data["av_change_abs_roll_mean_1000"]) * (((data["av_change_abs_roll_std_10"]) + (((np.tanh((data["avg_last_10000"]))) * (data["av_change_abs_roll_std_10"]))))))))) +
            0.0398984179*np.tanh(((data["kurt"]) * (((((((data["max_first_50000"]) * 2.0)) * 2.0)) * (((data["min_last_50000"]) * (((data["std_first_50000"]) - (((((-1.0*((data["mean_change_rate_first_50000"])))) + (((((data["min_roll_std_10"]) * 2.0)) * (np.tanh((data["max_first_50000"]))))))/2.0)))))))))) +
            0.0336784534*np.tanh(((data["q95_roll_mean_1000"]) - (((data["exp_Moving_average_30000_mean"]) + (((((data["av_change_abs_roll_std_1000"]) * 2.0)) * (((((((data["min_roll_std_10"]) * (((np.tanh((np.tanh((np.tanh((data["q95_roll_mean_1000"]))))))) * 2.0)))) + (data["min_roll_std_10"]))) - (np.tanh((data["max_to_min_diff"]))))))))))) +
            0.0397968367*np.tanh(((((data["classic_sta_lta3_mean"]) * (((((data["q05_roll_std_1000"]) - ((((((data["mean_change_abs"]) + (data["min_roll_std_10"]))/2.0)) + (((data["trend"]) * (((((data["q05_roll_std_1000"]) + (np.tanh((np.tanh((data["classic_sta_lta3_mean"]))))))) + (data["q99_roll_mean_1000"]))))))))) / 2.0)))) / 2.0)) +
            0.0398671627*np.tanh(((np.tanh((((data["iqr"]) * (((data["kurt"]) + (((((data["mean_change_rate_first_10000"]) + (data["mean_change_abs"]))) * (((((((((((data["max_to_min"]) + (data["q99_roll_mean_1000"]))) + (data["mean_change_rate_first_10000"]))/2.0)) + (data["med"]))/2.0)) + (data["mean_change_abs"]))))))))))) / 2.0)) +
            0.0393748768*np.tanh(((((data["min_roll_std_1000"]) - (((data["abs_max_roll_mean_10"]) - (data["count_big"]))))) * (((((data["skew"]) * (((data["std_roll_mean_10"]) + (data["abs_q99"]))))) + (((data["av_change_abs_roll_std_1000"]) * (((((data["q01_roll_std_1000"]) + (data["av_change_rate_roll_std_1000"]))) + (data["abs_max"]))))))))) +
            0.0259191245*np.tanh(((((((((data["Moving_average_6000_mean"]) + (((data["avg_last_50000"]) + (data["med"]))))) * (((((data["abs_q01"]) - (np.tanh((data["avg_last_50000"]))))) + (np.tanh((data["ave_roll_mean_1000"]))))))) - (np.tanh((data["avg_last_50000"]))))) + (data["med"]))) +
            0.0355381928*np.tanh(((((data["trend"]) * (((data["av_change_abs_roll_std_10"]) - (np.tanh((((data["med"]) + (((((((((((((((((data["av_change_abs_roll_std_100"]) * 2.0)) + (data["av_change_abs_roll_std_1000"]))) * 2.0)) + (data["av_change_abs_roll_std_10"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))))))) / 2.0)) +
            0.0399999991*np.tanh((-1.0*((((((data["q05_roll_std_1000"]) + (((data["med"]) * (data["q05_roll_std_1000"]))))) * (((data["q05_roll_std_1000"]) + (((((data["q05_roll_std_1000"]) + (((data["q05_roll_std_1000"]) + (((data["av_change_rate_roll_std_100"]) - (data["min_roll_std_1000"]))))))) - (data["min_roll_std_1000"])))))))))) +
            0.0398593470*np.tanh((((data["std_roll_mean_100"]) + (((np.tanh((((np.tanh((((((((((((data["min_roll_mean_1000"]) * 2.0)) * 2.0)) + (((((-1.0*((data["av_change_abs_roll_mean_100"])))) + (data["av_change_rate_roll_mean_100"]))/2.0)))) - (data["mean_change_rate_last_50000"]))) * 2.0)))) - (data["q99_roll_mean_100"]))))) + (data["min_roll_mean_1000"]))))/2.0)) +
            0.0399921872*np.tanh(((((data["med"]) - (np.tanh(((-1.0*((((((((((data["med"]) - (data["Moving_average_6000_mean"]))) * 2.0)) - (data["skew"]))) - ((-1.0*((data["av_change_abs_roll_mean_100"]))))))))))))) * (((data["av_change_rate_roll_mean_100"]) - (data["av_change_rate_roll_std_10"]))))) +
            0.0399999991*np.tanh(((np.tanh((((((((data["av_change_rate_roll_mean_100"]) + (((data["min_last_50000"]) - (data["min_first_10000"]))))) * (data["mean_change_rate_last_50000"]))) + (((data["av_change_abs_roll_std_100"]) * (((((data["av_change_rate_roll_mean_100"]) * (data["mean_change_rate_last_50000"]))) - (((data["min_roll_std_100"]) - (data["skew"]))))))))))) / 2.0)) +
            0.0399921872*np.tanh(((np.tanh((((((data["trend"]) * (((data["av_change_abs_roll_mean_10"]) + (((np.tanh((((data["av_change_rate_roll_mean_10"]) + (data["min_roll_std_1000"]))))) + (((data["q99_roll_std_100"]) * (((data["trend"]) + (((data["classic_sta_lta2_mean"]) + (data["av_change_rate_roll_mean_10"]))))))))))))) * 2.0)))) / 2.0)) +
            0.0399687439*np.tanh((((-1.0*((((data["trend"]) * (np.tanh((((((((data["classic_sta_lta4_mean"]) + (data["q05_roll_std_10"]))) + (np.tanh(((((-1.0*((data["skew"])))) + (((data["trend"]) * (data["classic_sta_lta4_mean"]))))))))) + ((-1.0*((data["av_change_abs_roll_std_100"]))))))))))))) / 2.0)) +
            0.0397812054*np.tanh(((((np.tanh((((((((((((np.tanh((np.tanh((np.tanh((((((data["q05"]) * 2.0)) * 2.0)))))))) + ((((data["ave_roll_mean_100"]) + (((data["ave_roll_mean_100"]) * (data["exp_Moving_average_3000_mean"]))))/2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) / 2.0)) +
            0.0397030637*np.tanh(((np.tanh((((np.tanh((((((((((((((((data["av_change_rate_roll_std_10"]) * (((data["min_first_50000"]) * (data["q99_roll_mean_1000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) + ((((np.tanh((data["av_change_rate_roll_std_100"]))) + (data["mean_change_rate_last_50000"]))/2.0)))))) / 2.0)) +
            0.0361711271*np.tanh(np.tanh((((data["max_last_50000"]) * (((((((data["max_last_50000"]) + (((np.tanh((data["av_change_abs_roll_mean_10"]))) * (((data["av_change_abs_roll_mean_10"]) * (data["av_change_abs_roll_mean_10"]))))))/2.0)) + (((np.tanh((((data["av_change_abs_roll_mean_10"]) * 2.0)))) * (((((data["q95_roll_mean_1000"]) * 2.0)) * 2.0)))))/2.0)))))) +
            0.0399531126*np.tanh(((np.tanh((((((((((data["min_roll_mean_100"]) * 2.0)) * 2.0)) - (((data["std_last_50000"]) * (data["av_change_rate_roll_std_1000"]))))) - (((data["av_change_abs_roll_std_10"]) + (data["min_roll_std_1000"]))))))) * ((((data["av_change_rate_roll_std_100"]) + (((np.tanh((data["q05_roll_mean_10"]))) * (data["av_change_rate_roll_std_100"]))))/2.0)))) +
            0.0283180308*np.tanh(((np.tanh((((((((((data["classic_sta_lta1_mean"]) * (((np.tanh(((-1.0*((data["mean_change_rate_last_50000"])))))) - (((data["mean_change_rate_first_10000"]) - (((data["q01_roll_std_1000"]) - (((data["mean_change_rate_first_10000"]) - (data["av_change_abs_roll_mean_100"]))))))))))) * 2.0)) * 2.0)) * 2.0)))) / 2.0)) +
            0.0398671627*np.tanh(((data["std_first_10000"]) * (((((data["av_change_abs_roll_std_1000"]) * ((((((data["mean_change_rate_last_10000"]) + (((((((data["std_first_10000"]) + (((((data["mean_change_abs"]) * 2.0)) - (data["q05_roll_std_100"]))))/2.0)) + ((((-1.0*((data["mean_change_rate_last_50000"])))) - (data["q95_roll_std_100"]))))/2.0)))/2.0)) / 2.0)))) / 2.0)))) +
            0.0377651863*np.tanh(((data["mean_change_rate_last_50000"]) * ((((np.tanh((((np.tanh(((((((data["classic_sta_lta3_mean"]) + (data["classic_sta_lta2_mean"]))/2.0)) * 2.0)))) + (data["min_roll_std_10"]))))) + (np.tanh((((data["mean_change_rate_last_50000"]) * (((data["mean_change_rate_last_50000"]) * (data["min_last_10000"]))))))))/2.0)))) +
            0.0356397703*np.tanh(np.tanh(((-1.0*((((((((data["classic_sta_lta1_mean"]) + (((data["ave_roll_std_1000"]) * (data["trend"]))))) + (((((data["ave_roll_std_1000"]) + (((data["skew"]) * (data["q95_roll_std_100"]))))) * (data["skew"]))))) * (((data["trend"]) * (data["skew"])))))))))) +
            0.0399843715*np.tanh(((data["q01_roll_std_1000"]) * (np.tanh((((((data["classic_sta_lta4_mean"]) * (((((data["mean_change_rate"]) - (((((data["classic_sta_lta2_mean"]) - (data["q99_roll_mean_100"]))) - (data["q99_roll_mean_100"]))))) - (((data["mean_change_abs"]) - (data["mean_change_rate"]))))))) + (data["classic_sta_lta4_mean"]))))))) +
            0.0398984179*np.tanh(((data["q01_roll_std_10"]) * (((data["av_change_rate_roll_mean_100"]) * (((data["max_first_50000"]) * ((((data["iqr"]) + ((((((data["q999"]) * ((((data["min_first_10000"]) + (data["av_change_rate_roll_mean_100"]))/2.0)))) + (((data["mean_change_rate_last_50000"]) * (data["av_change_rate_roll_mean_100"]))))/2.0)))/2.0)))))))) +
            0.0344207846*np.tanh(((np.tanh((((((((((data["max_to_min"]) + (data["mean_change_rate_last_50000"]))) + (((np.tanh(((((-1.0*((((data["max_to_min"]) + (((data["min_first_10000"]) + (((data["mean_change_rate_last_50000"]) + (data["max_to_min"])))))))))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)))) / 2.0)) +
            0.0393905044*np.tanh(np.tanh(((-1.0*(((((data["mad"]) + (((data["q01_roll_mean_10"]) - ((-1.0*(((((((data["mean_change_rate_first_50000"]) + (((data["mean_change_rate_last_50000"]) + ((((((data["mean_change_rate_first_50000"]) + (data["trend"]))) + (((data["max_first_10000"]) + (data["min_roll_std_100"]))))/2.0)))))/2.0)) / 2.0))))))))/2.0))))))) +
            0.0399999991*np.tanh(((((data["avg_first_10000"]) - (data["avg_first_50000"]))) * (((((((((((data["avg_first_50000"]) * (((data["classic_sta_lta1_mean"]) * (data["av_change_rate_roll_mean_10"]))))) * (data["avg_first_50000"]))) - (data["av_change_rate_roll_mean_10"]))) * (data["abs_max_roll_std_100"]))) - (((data["av_change_rate_roll_mean_10"]) * (data["q95_roll_std_1000"]))))))) +
            0.0350146517*np.tanh(((((data["std_roll_mean_1000"]) * (((((((data["kurt"]) * 2.0)) + (((((((((data["max_last_10000"]) * (((data["min_roll_mean_1000"]) * (data["min_roll_std_10"]))))) + (((data["trend"]) * 2.0)))/2.0)) + (data["trend"]))/2.0)))) + (data["av_change_abs_roll_std_100"]))))) / 2.0)) +
            0.0331470966*np.tanh(np.tanh((((((data["av_change_abs_roll_mean_10"]) * (((np.tanh(((((data["min_first_50000"]) + (((((data["skew"]) - (((data["skew"]) * (data["skew"]))))) - (data["mean_change_rate_first_50000"]))))/2.0)))) - ((-1.0*((((data["skew"]) * (data["trend"])))))))))) * 2.0)))) +
            0.0368353203*np.tanh(((data["mean_change_rate_first_50000"]) * (((data["std_roll_mean_1000"]) * (((data["av_change_rate_roll_std_100"]) * (((((data["q05_roll_mean_10"]) + (np.tanh((((np.tanh((((data["mean_change_rate_first_10000"]) - (data["q05_roll_mean_10"]))))) + (data["mean_change_rate_last_50000"]))))))) + (((data["mean_change_rate_first_10000"]) - (data["iqr"]))))))))))) +
            0.0391013846*np.tanh(((np.tanh((np.tanh((((((data["av_change_rate_roll_std_10"]) * (data["min_last_10000"]))) * (np.tanh((((((data["min_roll_std_1000"]) * 2.0)) * 2.0)))))))))) - (np.tanh((((data["av_change_abs_roll_std_100"]) * (((data["max_to_min_diff"]) * (((((data["q01_roll_std_1000"]) * 2.0)) * 2.0)))))))))) +
            0.0399765596*np.tanh(np.tanh(((((np.tanh((((((data["av_change_abs_roll_mean_10"]) * (data["classic_sta_lta4_mean"]))) - (np.tanh(((((data["trend"]) + (((data["q95_roll_mean_1000"]) * (data["min_roll_std_100"]))))/2.0)))))))) + (((((data["trend"]) * (data["min_roll_std_100"]))) * (((data["std_roll_std_10"]) * 2.0)))))/2.0)))) +
            0.0399453007*np.tanh(((data["av_change_abs_roll_std_100"]) * (((((((((((data["max_to_min_diff"]) + (data["Moving_average_6000_mean"]))/2.0)) - (((((((data["av_change_abs_roll_std_100"]) * (np.tanh((np.tanh(((((data["max_to_min_diff"]) + (data["exp_Moving_average_30000_mean"]))/2.0)))))))) * (data["av_change_abs_roll_std_100"]))) / 2.0)))) + (data["skew"]))/2.0)) / 2.0)))) +
            0.0399765596*np.tanh(((data["av_change_abs_roll_mean_10"]) * (((data["max_first_10000"]) * (((((((data["av_change_rate_roll_std_10"]) + ((-1.0*((data["max_first_10000"])))))) + ((((data["med"]) + ((-1.0*((data["iqr"])))))/2.0)))) + (((data["std_first_10000"]) * (data["max_first_10000"]))))))))) +
            0.0399531126*np.tanh(((np.tanh((((data["mean_change_abs"]) * (((np.tanh((((data["mean_change_abs"]) * (data["av_change_abs_roll_mean_10"]))))) + (((data["mean_change_rate_last_50000"]) - (((((data["av_change_abs_roll_mean_10"]) - (data["mean_change_rate_last_50000"]))) * (((((data["q99_roll_std_100"]) * 2.0)) * 2.0)))))))))))) / 2.0)))

In [ ]:
mean_absolute_error(y_tr,GPI(X_train_scaled)) 

In [ ]:
submission.time_to_failure = GPI(X_test_scaled).values
submission.to_csv('gpsubmission.csv',index=True)
submission.head()